## <center>Avaliação regular do perfil do corpo docente do PPCS IRR/Fiocruz <br /> Programa de Pós-graduação em Ciências da Saúde <br /> Instituto René Rachou – Fiocruz Minas</center>

    Rubens Lima do Monte Neto – Instituto René Rachou
    Antonio Marcos Aires Barbosa – Fiocruz Ceará
    Equipe do PPGCS

**Objetivo:**

    Reavaliação de meio termo do corpo docente do programa, a fim de acompanhar a manutenção dos parâmetros exigidos pela CAPES – Área de Medicina II e readequar a composição do grupo. São considerados os docentes permanentes (DP) e docentes colaboradores (DC), com base nos mesmos parâmetros

# Apuração do Indicadores de Gestão do PPGCS
### 1. Índice de Produção Conjunta dos docentes com discentes (IPC >= 50%):
    Indicador: Índice de publicações com discentes por orientador (IPC)
     Objetivo: Em, pelo menos, 50 % dos artigos publicados deve constar discentes do programa
         Meta: IPC >= 50,00
      
      Cálculo:

$$\sum_{k=1}^{n}\, 100 * \frac{QPCD}{QPAT}$$

        onde:
               n = Artigos completos publicado em periódicos indexados
            QPCD = Qte. Publicação de Artigos com discentes do Programa na lista de autores
            QPAT = Qte. Publicação de Artigos Total no período avaliado

### 2. Pontuação segundo Fator de Impacto (PFI >= 600 em pelo menos 70% dos docentes):

    Indicador: Total de pontos conforme periódicos das publicações no período
     Objetivo: Publicar trabalhos em periódicos de elevado impacto
               
         Meta: 70% dos docentes permanentes com PFI >= 600 pontos no quadriênio
               (150/ano e ao menos 03 artigos A, no mínimo 02 em A1, ou 04 artigos A2);
 
      Cálculo: Soma ponderada pela estratificação Qualis de acordo com o que segue:
        A1 = 100 pontos
        A2 = 80 pontos
        B1 = 60 pontos
        B2 = 40 pontos
        B3 = 20 pontos
        B4 = 10 pontos
        B5 = 2 pontos.


Parâmetro para classificaçao do periódico:

        A1: Periódicos com FI ou CPD >= 4,300
        A2: Periódicos com FI ou CPD entre 2,950 e 4,299
        B1: Periódicos com FI ou CPD entre 1,800 e 2,949
        B2: Periódicos com FI ou CPD entre 1,100 e 1,799
        B3: Periódicos com FI ou CPD entre 0,300 e 1,099
        B4: Periódicos com FI ou CPD entre 0,001 e 0,299, (ou Scielo, Scimago, PubMed ou Web of Science)
        B5: Periódicos sem FI ou CPD e indexado nas lases Lilacs ou Latindex    

## Grupos de Funções

In [1]:
# !pip3 install levenshtein
# !pip3 install editdistance
# !pip3 install seaborn
# !pip3 install pyjarowinkler

import re
import os
import time
import json
import numpy as np
import pandas as pd
import time # from datetime import time
from unidecode import unidecode
from datetime import timedelta
from datetime import datetime as dt
from string import Formatter
from datetime import date
from pyjarowinkler.distance import get_jaro_distance
from IPython.display import clear_output, display, HTML
pd.set_option('display.max_colwidth', None)
pd.set_option('colheader_justify', 'left')

def agrupar_orientacoes(filename):
    # Caminho para o arquivo JSON
    pathjson = '.\_data\in_json' 
    pathfilename = os.path.join(pathjson,filename)

    # dict_orientacoes = pd.read_json(pathfilename)

    # Carregar o JSON e aplicar a função
    with open(pathfilename, 'r', encoding='utf-8') as file:
        data = json.load(file)

    linhas_achatadas = extrair_orientacoes(data)
    df = pd.DataFrame(linhas_achatadas)

    # Salvar os dados achatados em um CSV
    df.to_csv('orientacoes_achatadas.csv', index=False)

    # Use .loc para modificar o DataFrame diretamente e evitar o aviso
    df.loc[:, 'sigla_natureza'] = df['natureza'].apply(lambda x: sigla_natureza(x))
    df_orientacoes = df[['id_lattes_orientadores','ano','sigla_natureza']]

    # Agrupar por 'ano', 'natureza' e orientador e contar as ocorrências
    contagem_orientacoes = df_orientacoes.groupby(['ano','sigla_natureza','id_lattes_orientadores']).size().reset_index(name='contagem')

    # Retornar a contagem de orientações por tipo e por ano
    return contagem_orientacoes

# Prosseguir com a definição e uso da função para extrair as orientações
def extrair_orientacoes(json_data):
    colunas = ['id_lattes_orientadores', 'tipo_orientacao', 'natureza', 'titulo', 'idioma', 'ano', 'id_lattes_aluno', 'nome_aluno', 'instituicao', 'pais', 'curso', 'codigo_do_curso', 'bolsa', 'agencia_financiadora', 'codigo_agencia_financiadora', 'nome_orientadores', 'tags', 'Hash']
    linhas = []

    # Iterar sobre cada tipo de orientação e ano
    for tipo_orientacao, anos in json_data.items():
        for ano, orientacoes in anos.items():
            for orientacao in orientacoes:
                # Preparar um dicionário para cada linha de acordo com as colunas definidas
                linha = {}
                # Adicionar dados específicos da orientação
                for campo in orientacao:
                    if campo in colunas:
                        linha[campo] = orientacao[campo]
                # Tratar 'id_lattes_orientadores' como uma lista, juntar os IDs com vírgula se houver mais de um
                linha['id_lattes_orientadores'] = ', '.join(orientacao.get('id_lattes_orientadores', []))
                # Adicionar o tipo de orientação como uma coluna
                linha['tipo_orientacao'] = tipo_orientacao
                # Adicionar o ano, garantindo que sobreponha qualquer valor de 'ano' nos dados individuais
                linha['ano'] = ano
                linhas.append(linha)

    return linhas

def sigla_natureza(natureza, separador=""):
    """
    Função para converter os valores da coluna 'natureza' em siglas, desconsiderando preposições,
    fazendo split de 'natureza' em palavras também pelo caractere "_".
    
    Parâmetros:
    - natureza: O valor da coluna 'natureza'.
    - separador: O caractere para separar as iniciais. Padrão é "_".
    
    Retorna:
    - Uma string que é a sigla formada pelas iniciais das palavras em 'natureza', desconsiderando preposições.
    """
    # Lista de preposições para serem ignoradas
    preposicoes = ['e', 'de', 'do', 'da', 'dos', 'das', 'em', 'na', 'no', 'nas', 'nos', 'por', 'para', 'com']

    # Mapeamento de exemplo conforme as siglas fornecidas
    mapeamento_siglas = {
        "OON": 1,
        "IC": 2,
        "MCCAE": 3,
        "SPD": 4,
        "TCCG": 5,
        "DM": 6,
        "TD": 7,
    }

    # Fazendo split por espaço e por underscore
    palavras = natureza.replace('_', ' ').replace('-', ' ').split()

    # Inicializar a lista de iniciais
    iniciais = []

    for palavra in palavras:
        # Verificar se a palavra não é uma preposição e não está vazia
        if palavra.lower() not in preposicoes and palavra:
            # Adicionar a inicial em maiúsculo à lista de iniciais
            iniciais.append(palavra[0].upper())

    # Juntar as iniciais para formar a sigla
    sigla = "".join(iniciais)

    # Aplicar o mapeamento para encontrar o número correspondente à sigla
    numero = mapeamento_siglas.get(sigla, "Desconhecido")

    # Retornar a string formatada com o número e a sigla
    return f"{numero}-{sigla}"

import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plotar_orientacoes_barras_agrupadas(contagem_orientacoes):
    # Supondo que 'contagem_orientacoes' seja o DataFrame com dados organizados por tipo e ano

    # Criar uma figura com subplots
    fig = make_subplots(rows=1, cols=1)

    # Encontrar anos e naturezas únicos para as orientações
    anos = sorted(contagem_orientacoes['ano'].unique())

    # Mapeamento de exemplo conforme as siglas fornecidas
    mapeamento_siglas = {
        "OON": 1,
        "IC": 2,
        "MCCAE": 3,
        "SPD": 4,
        "TCCG": 5,
        "DM": 6,
        "TD": 7,
    }

    # Ordenar as naturezas com base nos números associados em mapeamento_siglas
    mapeamento_siglas_para_numeros = {sigla: int(sigla.split('-')[0]) for sigla in contagem_orientacoes['sigla_natureza'].unique()}
    naturezas_ordenadas = sorted(contagem_orientacoes['sigla_natureza'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))

    # Criar uma barra para cada tipo de orientação em cada ano
    for natureza in naturezas_ordenadas:
        contagem_por_ano = []
        for ano in anos:
            # Somar as contagens para cada ano e natureza
            contagem = contagem_orientacoes[(contagem_orientacoes['ano'] == ano) & (contagem_orientacoes['sigla_natureza'] == natureza)]['contagem'].sum()
            contagem_por_ano.append(contagem)
        
        # Adicionar a barra ao gráfico
        fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza))

    # Atualizar o layout para permitir barras agrupadas
    fig.update_layout(barmode='group', title_text='Contagem de Orientações no Programa por Tipo e Ano', xaxis_title="Ano", yaxis_title="Quantidade de Orientações")

    # Mostrar o gráfico
    fig.show()


def plotar_orientacoes_barras_empilhadas(contagem_orientacoes):
    # 'contagem_orientacoes' DataFrame com a contagem por tipos por ano
    
    # Criar uma figura com subplots
    fig = make_subplots(rows=1, cols=1)

    # Encontrar anos únicos para as orientações
    anos = sorted(contagem_orientacoes['ano'].unique())

    # Mapeamento de exemplo conforme as siglas fornecidas
    mapeamento_siglas = {
        "OON": 1,
        "IC": 2,
        "MCCAE": 3,
        "SPD": 4,
        "TCCG": 5,
        "DM": 6,
        "TD": 7,
    }

    # Mapeamento de exemplo conforme as siglas fornecidas
    mapeamento_siglas_para_numeros = {
        "1-OON": 1,
        "2-IC": 2,
        "3-MCCAE": 3,
        "4-SPD": 4,
        "5-TCCG": 5,
        "6-DM": 6,
        "7-TD": 7,
    }

    # Ordenar as naturezas baseando-se no número extraído
    naturezas_ordenadas = sorted(contagem_orientacoes['sigla_natureza'].unique(), key=lambda x: mapeamento_siglas_para_numeros.get(x, 999))

    # Criar uma barra para cada tipo de orientação em cada ano, seguindo a ordem definida
    for natureza in naturezas_ordenadas:
        contagem_por_ano = []
        for ano in anos:
            # Somar as contagens para cada ano e natureza
            contagem = contagem_orientacoes[(contagem_orientacoes['ano'] == ano) & (contagem_orientacoes['sigla_natureza'] == natureza)]['contagem'].sum()
            # Se a sigla estiver associada ao número 1 ou 2, tornar a contagem negativa
            if mapeamento_siglas_para_numeros.get(natureza, 999) in [1, 2, 3, 4]:
                contagem = -contagem
            contagem_por_ano.append(contagem)
        
        # Adicionar a barra ao gráfico
        fig.add_trace(go.Bar(x=anos, y=contagem_por_ano, name=natureza))

    # Atualizar o layout para permitir barras empilhadas e ajustar o eixo Y para mostrar valores negativos
    fig.update_layout(
        barmode='relative',  # Usar 'relative' para empilhar incluindo valores negativos
        title_text='Contagem de Orientações por Tipo e Ano (curta duração abaixo do eixo X)',
        xaxis_title="Ano",
        yaxis_title="Quantidade de Orientações",
        yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor='black'),  # Destacar a linha zero
    )

    # Mostrar o gráfico
    fig.show()

In [2]:
pathjson = '.\_data\in_json' 
os.listdir(pathjson)

['642.files',
 '644.files',
 '863.advise.json',
 '863.files.zip',
 '863.graph.json',
 '863.list.json',
 '863.profile.json',
 '863.publication.json',
 '863patents.json',
 '863researchers_by_area.json',
 '863vosviewer.json',
 'unified_pub.json']

In [3]:
filename = '863.advise.json'
contagem_orientacoes = agrupar_orientacoes(filename)

In [4]:
plotar_orientacoes_barras_agrupadas(contagem_orientacoes)
plotar_orientacoes_barras_empilhadas(contagem_orientacoes)

In [ ]:
list_orientador=[]
for i, j in list(df['nome_orientadores'].items()):
    list_orientador.append(j[0])

ds_orientacoes = pd.Series(list_orientador)
ds_orientacoes.value_counts()

In [ ]:
df

In [ ]:
pathcsv = '.\_data\powerbi' 
os.listdir(pathcsv)

# Demais passos

Calcular índice de publicação em conjunto com alunos do programa

- Levantar nome dos discentes do programa
- Levantar os nomes de autores nas publicações de docentes
- Identificar por similadidade as publicações onde constam nome de alunos do programa

    Apurar colaborações docente/discente

In [ ]:
def converter_lista_set(lista):
    set1 = set(lista)
    return set1

def jaccard_similarity(set1, set2):
    '''
    Recebe dois conjuntos (sets) como entradas e retorna a similaridade Jaccard entre eles e avalia: 
    1. calcula a interseção dos dois conjuntos usando a função de interseção 
    2. calcula a união dos dois conjuntos usando a função de união 
    3. retorna a razão entre o comprimento da interseção e o comprimento da união, que é a similaridade de Jaccard.
    '''
    intersection = set1.intersection(set2)
    union        = set1.union(set2)
    return len(intersection) / len(union)
    
def montardf_docentes(lista_nomes1=False, lista_nomes2=False):
    # print(lista_nomes1)
    # print(lista_nomes2)

    ## Montagem do dataframe de participação docente
    if (lista_nomes1 and lista_nomes2) != False:
        ## Criar dataframe com os nomes do grupo de docentes permanentes
        file_path = os.path.join(pathcsv,'lista_docentes_permanentes.csv')
        df_docentes_permanentes   = pd.read_csv(file_path, header=None)
        # df_docentes_permanentes['GRUPO']='Permanente'
        # try:
        #     df_docentes_permanentes.drop(columns=([1,2]), inplace=True)
        # except:
        #     pass
        # df_docentes_permanentes.columns = ['DOCENTE','GRUPO']
        df_docentes_permanentes.columns = ['DOCENTE','IDLATES','PROGRAMA','GRUPO']

        ## Criar dataframe com os nomes do grupo de docentes colaboradores
        file_path = os.path.join(pathcsv,'lista_docentes_colaboradores.csv')
        df_docentes_colaboradores = pd.read_csv(file_path, header=None)
        # df_docentes_colaboradores['GRUPO']='Colaborador'
        # try:
        #     df_docentes_colaboradores.drop(columns=([1,2]), inplace=True)
        # except:
        #     pass
        df_docentes_colaboradores.columns = ['DOCENTE','IDLATES','PROGRAMA','GRUPO']

        ## Criar um dataframe único com todos grupos de docentes juntos
        df_docentes = pd.concat([df_docentes_permanentes, df_docentes_colaboradores]).reset_index(drop=True)
        return df_docentes

    elif 'permanentes' in lista_nomes1.lower():
        ## Criar dataframe com os nomes do grupo de docentes permanentes
        file_path = os.path.join(pathcsv,lista_nomes1)       
        df_docentes_permanentes   = pd.read_csv(file_path, header=None)
        # df_docentes_permanentes['GRUPO']='Permanente'
        # try:
        #     df_docentes_permanentes.drop(columns=([1,2]), inplace=True)
        # except:
        #     pass
        df_docentes_permanentes.columns = ['DOCENTE','IDLATES','PROGRAMA','GRUPO']
        return df_docentes_permanentes

    elif 'colaboradores' in lista_nomes1.lower():
        ## Criar dataframe com os nomes do grupo de docentes colaboradores
        file_path = os.path.join(pathcsv,lista_nomes1)
        df_docentes_colaboradores = pd.read_csv(file_path, header=None)
        # df_docentes_colaboradores['GRUPO']='Colaborador'
        # try:
        #     df_docentes_colaboradores.drop(columns=([1,2]), inplace=True)
        # except:
        #     pass
        df_docentes_colaboradores.columns = ['DOCENTE','IDLATES','PROGRAMA','GRUPO']
        return df_docentes_colaboradores
    else:
        print('Erro ao montar dataframe de docentes, verifique os nomes de arquivo.')
        return
        
def montar_listas(lista_csv, csv_permanentes=None, csv_colaboradores=None):
    if csv_permanentes == None and csv_colaboradores == None:
        csv_permanentes   = 'lista_docentes_permanentes.csv'
        csv_colaboradores = 'lista_docentes_colaboradores.csv'
        print(f'\nNomes de docentes não informados, utilizando caminho e nomes padrão:')
        print(f'     Docentes   permanentes de {pathcsv}{csv_permanentes}')
        print(f'     Docentes colaboradores de {pathcsv}{csv_colaboradores}')
        try:
            file_path = os.path.join(pathcsv,csv_permanentes)
            df_docperm = pd.read_csv(file_path, header=None)
            file_path = os.path.join(pathcsv,csv_colaboradores)
            df_docclbr = pd.read_csv(file_path, header=None)
            lista_docentes = pd.concat([df_docperm, df_docclbr], ignore_index=True)[0].values
            print(f'{len(lista_docentes):4} docentes permanentes e colaboradores encontrados')
        except Exception as e:
            print(f'Erro ao ler listas de docentes, verificar se os arquivos CSV estão na pasta {pathcsv}')
            print(e)
    elif 'permanentes' in csv_permanentes.lower():
        print(f'\nArquivo docentes   permanentes informado: {csv_permanentes}')
        try:
            file_path = os.path.join(pathcsv,csv_permanentes)
            lista_docentes = pd.read_csv(file_path, header=None)[0].values
            print(f'{len(lista_docentes)} docentes permanentes encontrados')
        except Exception as e:
            print(e)
    elif 'colaboradores' in csv_permanentes.lower():
        print(f'\nArquivo docentes colaboradores informado: {csv_colaboradores}')
        try:
            file_path = os.path.join(pathcsv,csv_permanentes)
            lista_docentes = pd.read_csv(file_path, header=None)[0].values
            print(f'{len(lista_docentes)} docentes colaboradores encontrados')
        except Exception as e:
            print(e)    
    else:
        print('Erro ao ler listas de docentes, verificar listas')

    df_prod   = montardf_producao(lista_csv)
    
    ## Montar a lista de autores com limpar_nomes remove caracteres, preposições e separa iniciais com espaço
    lista_listas = df_prod['AUTORES'].tolist()
    lista_autores_artigos = []
    for i in lista_listas:
        lista_autores_artigos.append(limpar_nomes(i))
    
    ## Ler nomes de discentes e orientadores
    lista_orientadores, lista_discentes = ler_lista_orientacoes()
    
    return lista_autores_artigos, lista_docentes, lista_orientadores, lista_discentes

def montardf_participacao_docente_discente(df_prod, dic_nomes_docentes, dic_nomes_discentes):
    ## Montar dataframe de participação docente
    df_participacao_docente = pd.DataFrame(dic_nomes_docentes).T
    df_participacao_docente.columns = ['DOCENTE','INDICES_ARTIGOS']
    df_participacao_docente['AUTORIAS'] = [len(x) for x in df_participacao_docente['INDICES_ARTIGOS']]

    ## Montar dataframe de participação discente
    df_participacao_discente = pd.DataFrame(dic_nomes_discentes).T
    df_participacao_discente.columns = ['DISCENTE','INDICES_ARTIGOS']
    df_participacao_discente['AUTORIAS'] = [len(x) for x in df_participacao_discente['INDICES_ARTIGOS']]

    ## Criar lista com os artigos onde foi encontado nome de algum docente
    artigos_com_docentes=[]
    for m in df_participacao_docente['INDICES_ARTIGOS']:
        for n in m:
            if n not in artigos_com_docentes:
                artigos_com_docentes.append(n)
    artigos_com_docentes.sort()
                
    ## Criar lista com os artigos onde foi encontado nome de algum discente
    artigos_com_discentes=[]
    for m in df_participacao_discente['INDICES_ARTIGOS']:
        for n in m:
            if n not in artigos_com_discentes:
                artigos_com_discentes.append(n)
    artigos_com_discentes.sort()
                
        
    ## Criar lista com os artigos onde NÃO foi encontado nome de discente
    lista_semparticipacaodiscente=[]
    for i in range(len(df_prod.index)):
        if i not in artigos_com_discentes and i not in lista_semparticipacaodiscente:
            lista_semparticipacaodiscente.append(i)
            
    ## Criar lista com os artigos onde NÃO foi encontado nome de docente
    lista_semparticipacaodocente=[]
    for i in range(len(df_prod.index)):
        if i not in artigos_com_docentes and i not in lista_semparticipacaodocente:
            lista_semparticipacaodocente.append(i)

    ## Apresentar resultados das buscas por nomes de autores docentes e discentes
    lista_titulos = pd.Series(df_prod['TITULO'].values).unique().tolist()
    print(len(lista_titulos),'títulos únicos de artigo encontrados')
    pdoc = np.round(100*len(artigos_com_docentes)/len(df_prod.index),2)
    spdoc = np.round(100*len(lista_semparticipacaodocente)/len(df_prod.index),2)
    print(f'{len(artigos_com_docentes):4} ({pdoc:6}%) artigos com nome de  docente encontrado, faltando {len(lista_semparticipacaodocente):3} ({spdoc:5}%)')

    pdis = np.round(100*len(artigos_com_discentes)/len(df_prod.index),2)
    spdis = np.round(100*len(lista_semparticipacaodiscente)/len(df_prod.index),2)
    print(f'{len(artigos_com_discentes):4} ({pdis:6}%) artigos com nome de discente encontrado, faltando {len(lista_semparticipacaodiscente):3} ({spdis:5}%)')

    return df_participacao_docente, df_participacao_discente, lista_semparticipacaodocente, lista_semparticipacaodiscente

    Ler arquivos de dados do disco local

In [ ]:
## ler arquivos de publicação de artigos na pasta de arquivos CSV
def ler_artigostodosperiodos():    
    print(pathcsv)
    import os, sys

    lista_csv=[]
    dirs = os.listdir(pathcsv)
    for file in dirs:
        if 'Artigos' in file:
            lista_csv.append(file)
    lista_csv.sort()
    
    for i in lista_csv:
        print(i)

    return lista_csv



## ler arquivos de publicação de de período determinado
def ler_csvptg(inicio=False, final=False, tipo=False, grupo=False):    
    import os, sys

    print(pathcsv)
    lista_csv=[]
    dirs = os.listdir(pathcsv)
    for file in dirs:
        if (str(inicio) or str(final)) == False:
            if tipo.lower() == False:
                if grupo.lower() == False:
                    lista_csv.append(file)

        elif (str(inicio) and str(final)) in file.lower():
            if unidecode(tipo).lower() in unidecode(file).lower():
                if unidecode(grupo).lower() in unidecode(file).lower():
                    lista_csv.append(file)
    lista_csv.sort()
    
    for i in lista_csv:
        print(i)

    return lista_csv



def ler_nomesdocentes():    
    print(pathcsv)
    import os, sys

    lista_csv=[]
    dirs = os.listdir(pathcsv)
    for file in dirs:
        if 'nomes_docentes' in file:
            lista_csv.append(file)
    lista_csv.sort()
    
    for i in lista_csv:
        print(i)

    return lista_csv


## ler arquivo com as orientações para gerar a lista de discentes
def ler_lista_orientacoes():
    try:
        l1='lista_orientadores-discentes.csv'
        df_orientacoes = pd.read_csv(pathcsv+l1, delimiter=';', header=None)
        
        lista_orientadores = df_orientacoes.iloc[:,0].unique()
        lista_discentes    = df_orientacoes.iloc[:,1].unique()
        print(f'{len(lista_orientadores):4} orientadores, com {len(lista_discentes)} discentes encontrados')
    except Exception as e:
        print('Erro ao gerar lista de orientações:')
        print(e)
        return df_orientacoes
        
    return lista_orientadores, lista_discentes 


## montar um dataframe com nome dos discentes de cada orientador
def montardf_orientacoes():
    try:
        l1='lista_orientadores-discentes.csv'
        df_orientacoes = pd.read_csv(pathcsv+l1, delimiter=';', header=None)
        df_orientacoes.columns=['ORIENTADOR','DISCENTE']
        
    except Exception as e:
        print('Erro ao dividir dataframe de orientações:')
        print(e)
        return
        
    return df_orientacoes



def montardf_docentes_permanentes_colaboradores():
    try:
        l1='lista_docentes_colaboradores.csv'
        l2='lista_docentes_permanentes.csv'
        df_docclbr = pd.read_csv(os.path.join(pathcsv,l1), header=None)
        df_docperm = pd.read_csv(os.path.join(pathcsv,l2), header=None)
        df_docentes = pd.concat([df_docperm, df_docclbr], ignore_index=True)
        print(f'{len(df_docentes.index):4} docentes permanentes e colaboradores encontrados')
    except Exception as e:
        print(e)
        
    return df_docentes



def montardf_producao(lista_csv):
    df_public=pd.DataFrame()
    for nome_csv in lista_csv:
        if 'colaboradores' in nome_csv.lower():
            tipo='colaboradores'
        else:
            tipo='permanentes'
        
        df_pub = pd.read_csv(pathcsv+nome_csv)

        pat='\t\t\t\t\t\t\t\t\t\t\t\t \t\t\t\t\t\t\t\t\t\t\t\t\t'
        df_temp1 = df_pub.Data.str.split(pat=pat,expand=True)
        df_temp1.columns = (['TITULO','RevAut'])

        pat1='\t\t\t\t\t\t\t\t\t\t\t\t \t\t\t\t\t\t\t\t\t\t\t\t'
        df_temp2 = df_temp1.RevAut.str.split(pat=pat1,expand=True)
        df_temp2.columns = (['REVISTA','AUTORES'])

        df_temp0 = df_pub.drop(['Data'], axis=1)
        df_pub=df_temp0.merge(df_temp1['TITULO'],left_index=True,right_index=True)
        df_pub=df_pub.merge(df_temp2,left_index=True,right_index=True)
        try:
            df_pub.drop(['Issn','Natureza'], axis=1, inplace=True)
        except:
            pass
        try:
            df_pub.drop(['Tipo','Idioma'], axis=1, inplace=True)
        except:
            pass

        df_public = pd.concat([df_public, df_pub], ignore_index=True)
        # print(len(df_public.index))
        
    ## Extrai o período com base nos dados
    inicio = min(df_public['Ano'])
    final  = max(df_public['Ano'])
    
    total_artigos=len(df_public.index)
    # print(f'{total_artigos:4} publicações de artigos de docentes do programa no período de {inicio} a {final} carregadas...') 
    
    return df_public

### Funções para avaliar a PCD e Pontuação de Impacto

In [ ]:
def montardf_impacto_docente_discente(df_prod, dic_nomes_docentes, dic_nomes_discentes, inicio, final):
    artigos_com_discentes=[]
    artigos_com_docentes=[]
    lista_semparticipacaodiscente=[]
    lista_semparticipacaodocente=[]

    try:
        ## Montar dataframe de participação docente
        df_impacto_docente = pd.DataFrame(dic_nomes_docentes).T
        df_impacto_docente.columns = ['DOCENTE','INDICES_ARTIGOS','EXTRATOS_QUALIS']
        df_impacto_docente['AUTORIAS'] = [len(x) for x in df_impacto_docente['INDICES_ARTIGOS']]

        ## Criar lista com os artigos onde foi encontado nome de algum docente
        for m in df_impacto_docente['INDICES_ARTIGOS']:
            for n in m:
                if n not in artigos_com_docentes:
                    artigos_com_docentes.append(n)
        artigos_com_docentes.sort()

        ## Criar lista com os artigos onde NÃO foi encontado nome de docente
        for i in range(len(df_prod.index)):
            if i not in artigos_com_docentes and i not in lista_semparticipacaodocente:
                lista_semparticipacaodocente.append(i)
    except:
        df_impacto_docente = pd.DataFrame()
        print('Não foi possível encontrar nenhuma ocorrência dos nomes dos docentes com este conjunto de dados')
        pass

    try:
        ## Montar dataframe de participação discente
        df_impacto_discente = pd.DataFrame(dic_nomes_discentes).T
        df_impacto_discente.columns = ['DISCENTE','INDICES_ARTIGOS','EXTRATOS_QUALIS']
        df_impacto_discente['AUTORIAS'] = [len(x) for x in df_impacto_discente['INDICES_ARTIGOS']]

        ## Criar lista com os artigos onde foi encontado nome de algum discente
        for m in df_impacto_discente['INDICES_ARTIGOS']:
            for n in m:
                if n not in artigos_com_discentes:
                    artigos_com_discentes.append(n)
        artigos_com_discentes.sort()
    except:
        df_impacto_discente = pd.DataFrame()
        print('Não foi possível encontrar nenhuma ocorrência dos nomes dos discentes com este conjunto de dados')
        pass
                
    ## Criar lista com os artigos onde NÃO foi encontado nome de discente
    for i in range(len(df_prod.index)):
        if i not in artigos_com_discentes and i not in lista_semparticipacaodiscente:
            lista_semparticipacaodiscente.append(i)

    ## Apresentar resultados das buscas por nomes de autores docentes e discentes
    lista_titulos = pd.Series(df_prod['TITULO'].values).unique().tolist()
    print(len(lista_titulos),'títulos únicos de artigo encontrados')
    pdoc  = np.round(100*len(artigos_com_docentes)/len(df_prod.index),2)
    spdoc = np.round(100*len(lista_semparticipacaodocente)/len(df_prod.index),2)
    print(f'{len(artigos_com_docentes):4} ({pdoc:6}%) artigos com nome de  docente encontrado, faltando {len(lista_semparticipacaodocente):3} ({spdoc:5}%)')

    pdis = np.round(100*len(artigos_com_discentes)/len(df_prod.index),2)
    spdis = np.round(100*len(lista_semparticipacaodiscente)/len(df_prod.index),2)
    print(f'{len(artigos_com_discentes):4} ({pdis:6}%) artigos com nome de discente encontrado, faltando {len(lista_semparticipacaodiscente):3} ({spdis:5}%)')

    ## A1=100, A2=80, B1=60, B2=40, B3=20, B4=10, B5=2
    soma_impacto=[]
    for linha in df_impacto_docente['EXTRATOS_QUALIS']:
        impacto=0
        for extrato in linha:
            if extrato == 'A1':
                impacto+=100
            elif extrato == 'A2':
                impacto+=80
            elif extrato == 'B1':
                impacto+=60
            elif extrato == 'B2':
                impacto+=40
            elif extrato == 'B3':
                impacto+=20
            elif extrato == 'B4':
                impacto+=10
            elif extrato == 'B5':
                impacto+=2
            elif extrato == 'C':
                impacto+=0
            elif extrato == 'nan':
                impacto+=0
        soma_impacto.append(impacto)

    df_impacto_docente['SOMA_IMPACTO'] = soma_impacto
    qte_anos = (final-inicio+1)
    df_impacto_docente['ANOS'] = qte_anos
    df_impacto_docente['IMPACTO_MEDIO_ANUAL'] = np.round(df_impacto_docente['SOMA_IMPACTO']/df_impacto_docente['ANOS'],1)

    return df_impacto_docente, df_impacto_discente



def apurar_pcd_impacto(df_docentes, df_impacto_docente, df_impacto_discente, meta_pcd=50.0, meta_impacto=150.0):
    print(f'Total de nomes de docentes em análise: {len(df_docentes.index)}')
    print(f'Total de nomes de docentes  encontrados nos artigos: {len(df_impacto_docente.index)}')
    print(f'Total de nomes de discentes encontrados nos artigos: {len(df_impacto_discente.index)}')

    df_docentes_pcd_impacto = df_docentes
    ## Montar lista com os índices do dataframe de artigos onde foram achados nomes de discentes na lista de autores
    lista_participacao_discente = []
    for artigos_discentes in df_impacto_discente['INDICES_ARTIGOS']:
        for indice in artigos_discentes:
            lista_participacao_discente.append(indice)

    ## Contar a quantidade de participações de discentes que ocorrem no dataframe de produção docente
    qte_colab_discente=[]
    for docente,artigos_docente in zip(df_impacto_docente['DOCENTE'], df_impacto_docente['INDICES_ARTIGOS']):
        qte=0
        for indice in artigos_docente:
            if indice in lista_participacao_discente:
                qte+=1
            
        qte_colab_discente.append(qte)

    df_docentes_pcd_impacto['PUBLICAÇÕES']  = df_impacto_docente['AUTORIAS']
    df_docentes_pcd_impacto['COM_DISCENTE'] = qte_colab_discente
    df_docentes_pcd_impacto['PCD'] = np.round(100*(df_docentes_pcd_impacto['COM_DISCENTE']/df_impacto_docente['AUTORIAS']),1)
    df_docentes_pcd_impacto['IMPACTO'] = df_impacto_docente['SOMA_IMPACTO']
    df_docentes_pcd_impacto['IMPACTO_MEDIO_ANUAL'] = df_impacto_docente['IMPACTO_MEDIO_ANUAL']

    ## Definir a meta de produção conjunta com discente e apurar o resultado
    # meta_pcd=50.0
    apuracao_pcd     = df_docentes_pcd_impacto.groupby([df_docentes_pcd_impacto.index,'PCD'])
    df_abaixo_pcd    = apuracao_pcd.filter(lambda x: x['PCD'] < meta_pcd)
    df_atingiram_pcd = apuracao_pcd.filter(lambda x: x['PCD'] >= meta_pcd)

    total_docentes         = len(df_docentes_pcd_impacto.index)

    contagem_abaixometa    = len(df_abaixo_pcd.index)
    contagem_atingindometa = len(df_atingiram_pcd.index)
    indicador_pcd = np.round(100*contagem_atingindometa/total_docentes,1)
    print(f'{indicador_pcd}% dos docentes {contagem_atingindometa}/{total_docentes} atingem a meta de {meta_pcd}% publicação com discente')

    ## Definir a meta de impacto por pesquisador e para o grupo
    # meta_impacto=150
    apuracao_impacto     = df_docentes_pcd_impacto.groupby([df_docentes_pcd_impacto.index,'IMPACTO'])
    df_abaixo_impacto    = apuracao_impacto.filter(lambda x: x['IMPACTO_MEDIO_ANUAL'] < meta_impacto)
    df_atingiram_impacto = apuracao_impacto.filter(lambda x: x['IMPACTO_MEDIO_ANUAL'] >= meta_impacto)

    contagem_abaixometa_impacto    = len(df_abaixo_impacto.index)
    contagem_atingindometa_impacto = len(df_atingiram_impacto.index)
    indicador_impacto = np.round(100*contagem_atingindometa_impacto/total_docentes,1)
    print(f'{indicador_impacto}% dos docentes {contagem_atingindometa_impacto}/{total_docentes} atingem a meta de {meta_impacto} pontos de impacto médio por ano das publicações')

    return df_docentes_pcd_impacto, apuracao_pcd, df_abaixo_pcd, df_atingiram_pcd, indicador_pcd, indicador_impacto

    
   
def mostrar_partes_achadas(lista_discentes, lista_autores, verbose=False):
    partes_achadas=[]
    discentes_achados=[]
    erros=[]
    if verbose == True:
        print(f'{padronizar_titulo(lista_autores).lower()}')
    try:
        ## Buscar pelas partes de nomes de autor em cada linha de autores de artigo
        for nome in lista_discentes:
            sobrenome, partenome1, partenome2, partenome3 = quebrar_partesnomes(nome)
            if verbose == True:
                print(f'{sobrenome}, {partenome1} {partenome2} {partenome3}')
            strbusca_partes = compilar_partes(sobrenome, partenome1, partenome2, partenome3)
            try:
                achados_partes = re.search(strbusca_partes, padronizar_titulo(lista_autores).lower())
                if achados_partes.span() !=None:
                    partes_achadas.append(achados_partes.groups())
                    discentes_achados.append(nome)
            except Exception as e:
                # print(e)
                pass

        ## Buscar pelas iniciais de partes de nomes de autor, que seguem um sobrenome em cada linha de autores de artigo
        for nome in lista_discentes:
            sobrenome, inicial1, inicial2, inicial3 = quebrar_iniciais(nome)
            if verbose == True:
                print(f'{sobrenome}, {inicial1} {inicial2} {inicial3}')
            strbusca_iniciais = compilar_iniciais(sobrenome, inicial1, inicial2, inicial3)
            try:
                achados_iniciais = re.search(strbusca_iniciais, padronizar_titulo(lista_autores).lower())
                if achados_iniciais.span() !=None and achados_iniciais.groups() not in discentes_achados:
                    partes_achadas.append(achados_iniciais.groups())
                    discentes_achados.append(nome)
            except Exception as e:
                # print(e)
                pass
    except Exception as e:
        # print(f'Erro ao buscar partes de nome; {e}')
        print(e)
    
    return partes_achadas, discentes_achados



def listar_achados(docente):
    lista_csv = ler_artigostodosperiodos()
    lista_autores_artigos, lista_docentes, lista_orientadores, lista_discentes = montar_listas(lista_csv)

    df_filtrado = df_impacto_docente[(df_impacto_docente.DOCENTE==docente)]

    lista_indices = df_filtrado['INDICES_ARTIGOS'].values.tolist()[0]
    n=100
    print('-'*n)
    print(f'\nDocente: {docente} | {len(lista_indices)} Publicações identificadas para no período [{inicio} a {final}]')
    print()
    for indice in lista_indices:
        print('-'*n)
        print(f'Índice da publicação: {indice}')
        print(df_public.iloc[indice].values[:3])
        lista_autores = padronizar_titulo(df_public.iloc[indice].values[4])
        partes_achadas, discentes_achados = mostrar_partes_achadas(lista_discentes, lista_autores, verbose=False)
        print()
        print(lista_autores)
        print('\nPartes de nomes de alunos encontrados na lista de autores:')
        print(partes_achadas)
        print('\nNomes de alunos considerados como encontrados na lista de autores:')
        print(discentes_achados)

### Funções para plotagem: 
    
    Plotar gráficos de percentual de participação discente

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from matplotlib.patches import Circle
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage, AnnotationBbox)
from matplotlib.cbook import get_sample_data
plt.rcParams['font.size']      = 12
# plt.rcParams["figure.figsize"] = (15,9)



def plotar_pcd(df, grupo=False, inicio=False, final=False):
    N    = len(df.index)
    percentual = (df['PCD'].values.round(1))

    ind   = np.arange(N) # the x locations for the groups
    width = 0.75         # the width of the bars: can also be len(x) sequence

    # criar figura
    fig, ax = plt.subplots(figsize=(25,10))

    # plotar barras verticais com cores condicionais se abaixo do valor da variável para aceitação
    par = 50 
    cor = ['yellow' if (x < par) else 'green' for x in percentual]
    p1  = ax.bar(ind, percentual, width, 
                #  yerr=dsvpad, 
                 error_kw=dict(lw=0.3, capsize=2, capthick=1),
                 label='Percentual de publicação com discente', color=cor)

    # plotar os rótulos e título
    ax.axhline(par, color='red', linewidth=3, linestyle='dotted')
    ax.axhline(70, color='gray', linewidth=3, linestyle='dotted')
    if (grupo and inicio and final) != False:
        ax.set_title(f'Apuração do percentual de publicação de docentes {grupo.upper()} com discente no período de {inicio} a {final}')
    elif grupo == False:
        ax.set_title(f'Apuração do percentual de publicação de docentes com discente no período de {inicio} a {final}')
    else:
        ax.set_title(f'Apuração do percentual de publicação com discente')

    ax.set_ylabel('Percentual de artigos publicados com discente')
    ax.set_xticks(ind)
    
    labels_pos=np.arange(1,N+1)
    ax.set_xticklabels(labels_pos)

    # Label with label_type 'center' instead of the default 'edge'
    ax.bar_label(p1, label_type='center')
    # ax.bar_label(p1, dsvpad)
    # ax.set_yticks(range(0,100))
    
    # respectivo domínio de cada questão no AGREE II
    grupos = ['01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01',
              '02','02','02','02','02','02','02','02','02','02','02','02','02','02','02','02',]
    
    # calcular limites de retângulos dos domínios
    lista_dominios   = pd.Series(grupos)
    largura_dominios = lista_dominios.value_counts().sort_index().values   
    altura = 100
    rets=[]   
    
    for i in range(1,len(largura_dominios)+1):
        ret = patches.Rectangle((-0.5,0),
                                np.sum(largura_dominios[:i]), altura,
#                                 linestyle='dashdot',
                                linewidth=2,
                                edgecolor='b',
                                fill = False)
        rets.append(ret)
    
    # plotar os retângulos das dimensões na área do gráfico
    for i in rets:
        ax.add_patch(i)
        
    # plotar legenda, comentar para excluir
    # ax.legend(bbox_to_anchor=(0.75,-0.05), ncol=2)
    
    campo='04'
    # savefig_respostas(campo)
    
    plt.show()
    return 



def plotar_medias_impacto(df, grupo=False, inicio=False, final=False):
    N      = len(df.index)
    pontos = (df['IMPACTO_MEDIO_ANUAL'].values.round(1))

    ind   = np.arange(N) # the x locations for the groups
    width = 0.75         # the width of the bars: can also be len(x) sequence

    # criar figura
    fig, ax = plt.subplots(figsize=(25,10))

    # plotar barras verticais com cores condicionais se abaixo do valor da variável para aceitação
    par = 150 
    cor = ['yellow' if (x < par) else 'green' for x in pontos]
    p1  = ax.bar(ind, pontos, width, 
                #  yerr=dsvpad, 
                 error_kw=dict(lw=0.3, capsize=2, capthick=1),
                 label='Pontuação em impacto das publicações de docentes', color=cor)

    # plotar os rótulos e título
    ax.axhline(par, color='red', linewidth=3, linestyle='dotted')
    if (grupo and inicio and final) != False:
        ax.set_title(f'Apuração da pontuação de impacto das publicações de docentes {grupo.upper()} no período de {inicio} a {final}')
    elif (inicio and final) != False:
        ax.set_title(f'Apuração da pontuação de impacto das publicações de docentes no período de {inicio} a {final}')
    else:
        ax.set_title(f'Apuração da pontuação de impacto médio (total do impacto acumulado / quantidade de anos do período) das publicações de docentes')

    ax.set_ylabel('Pontuação ponderada pelo Qualis dos artigos publicados')
    ax.set_xticks(ind)
    
    labels_pos=np.arange(1,N+1)
    ax.set_xticklabels(labels_pos)

    # Label with label_type 'center' instead of the default 'edge'
    ax.bar_label(p1, label_type='center')
    # ax.bar_label(p1, dsvpad)
    vr_maximo = int(max(pontos)+50)
    # ax.set_yticks(range(0,vr_maximo))
    
    # respectivo domínio de cada questão no AGREE II
    grupos = ['01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01','01',
              '02','02','02','02','02','02','02','02','02','02','02','02','02','02','02','02',]
    
    # calcular limites de retângulos dos domínios
    lista_dominios   = pd.Series(grupos)
    largura_dominios = lista_dominios.value_counts().sort_index().values   
    altura = max(pontos)+50
    rets=[]   
    
    for i in range(1,len(largura_dominios)+1):
        ret = patches.Rectangle((-0.5,0),
                                np.sum(largura_dominios[:i]), altura,
#                                 linestyle='dashdot',
                                linewidth=2,
                                edgecolor='b',
                                fill = False)
        rets.append(ret)
    
    # plotar os retângulos das dimensões na área do gráfico
    for i in rets:
        ax.add_patch(i)
        
    # plotar legenda, comentar para excluir
    # ax.legend(bbox_to_anchor=(0.75,-0.05), ncol=2)
    
    campo='04'
    # savefig_respostas(campo)
    
    plt.show()
    return 

### Funções avaliação PCD e Impacto Médio Anual: 
    Avaliar indicadores PCD e Somatório do Fator de Impacto

In [ ]:
def avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes):
    ## Ler arquivos de dados
    lista_csv   = ler_csvptg(inicio, final, tipo, grupo)
    df_public   = montardf_producao(lista_csv)
    df_docentes = montardf_docentes(lista_nomes_docentes)
    lista_autores_artigos, lista_docentes, lista_orientadores, lista_discentes = montar_listas(lista_csv, lista_nomes_docentes)

    ## Avaliação da Publicação Conjunta com Discentes (PCD) e do Impacto Médio Anual (IMA)
    dic_nomes_docentes, dic_nomes_discentes, docentes_naoencontrados, discentes_naoencontrados, erros = gerar_dicionarios(df_public, lista_docentes, lista_discentes, inicio, final)
    df_impacto_docente, df_impacto_discente = montardf_impacto_docente_discente(df_public, dic_nomes_docentes, dic_nomes_discentes, inicio, final)

    ## Gerar gráfico de apuração de impacto médio anual por docente
    df_docentes_pcd_impacto, apuracao_pcd, df_abaixo_pcd, df_atingiram_pcd, indicador_pcd, indicador_impacto = apurar_pcd_impacto(df_docentes, df_impacto_docente, df_impacto_discente)
    plotar_pcd(df_docentes_pcd_impacto, grupo, inicio, final)
    plotar_medias_impacto(df_docentes_pcd_impacto, grupo, inicio, final)

    return df_public, df_impacto_docente, df_impacto_discente, df_docentes_pcd_impacto, indicador_pcd, indicador_impacto

# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
# .
## Apurar participação discente e impacto artigos dos docentes
### Todos os períodos e grupos

In [ ]:
os.listdir(pathcsv)

In [ ]:
lista_nomes_docentes_permanentes   = 'lista_docentes_permanentes.csv'
lista_nomes_docentes_colaboradores = 'lista_docentes_colaboradores.csv'
df_docentes = montardf_docentes(lista_nomes_docentes_permanentes, lista_nomes_docentes_colaboradores)

## Escolha dos arquivos que alimentarão a análise da produção
lista_csv = ler_artigostodosperiodos()
lista_csv

In [ ]:
df_docentes

In [ ]:
df_prod   = montardf_producao(lista_csv)

## Mostrar quantitativos lidos
print(f'\nCarregado dataframe com {len(df_prod.index)} linhas:')
lista_titulos = pd.Series(df_prod['TITULO'].values).unique().tolist()
print(f'{len(lista_titulos):4} artigos distintos publicados no período')
lista_revistas = pd.Series(df_prod['REVISTA'].values).unique().tolist()
print(f'{len(lista_revistas):4} revistas distintas utilizadas no período')

## Ler nomes de docentes, discentes e papel de orientador
lista_autores_artigos, lista_docentes, lista_orientadores, lista_discentes = montar_listas(lista_csv)
qte_docentes=len(lista_docentes)
qte_discentes=len(lista_discentes)
total_iteracoes=(qte_docentes+qte_discentes)

### Testes

In [ ]:
# a,b='',''
# get_jaro_distance(a, b)

In [ ]:
a,b='1','0'
get_jaro_distance(a, b)

In [ ]:
lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
print(len(lista_nomes_autores))
print(lista_nomes_autores[0])

## Verifica se a divisão em nomes de autor é par (sobrenome separado de nomes por vírgula)
for i in lista_nomes_autores[:14]:
    qte_nomes_autor = len(i.split(','))
    if qte_nomes_autor/2 != qte_nomes_autor//2:
        print(qte_nomes_autor,'\n')

In [ ]:
for m,docente in enumerate(lista_docentes):
    nome_docente_padronizado = padronizar_nome(docente).lower()
    iniciais_nome_docente    = iniciais_nome(docente).lower()
    print(iniciais_nome_docente)

In [ ]:
    # ## Monta a lista de autores com a divisão da string de nomes acima por: sobrenome, nomes
    # lst_autores_artigo = []
    # lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
    # for n,nomes_autor in enumerate(lista_nomes_autores):
    #     autores,erros = organizar_nomes(nomes_autor)
    #     # print(autores,'\n')
    #     for o,autor in enumerate(autores):
    #         # print(autor)
    #         # print(f'{o+1:2}/{qte_autores_artigo:2} nomes de autor do artigo em análise, restando {qte_autores_artigo-o-1:2}')
    #         sobrenome_iniciais_autor = iniciais_nome(autor).lower()
    #         string_iniciais_autor    = ' '.join(x.strip() for x in sobrenome_iniciais_autor.split(',')[1:])
    #         set_iniciais_autor       = converter_lista_set([x.strip() for x in sobrenome_iniciais_autor.split(',')[1:]])
    #         print(f'{sobrenome_iniciais_autor:20}|{string_iniciais_autor:^9}|{set_iniciais_autor}')
    #     lst_autores_artigo.append(sobrenome_iniciais_autor)
    # print(f'Lista organizada de nomes de autores: {len(lst_autores_artigo)} listas de autores de artigos publicados no período')

In [ ]:
    qte_docentes     = len(lista_docentes)
    qte_discentes    = len(lista_discentes)
    total_iteracoes  = (qte_docentes+qte_discentes)
    
    ## Monta uma lista de strings com nomes dos autores de cada artigo
    lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
    qte_artigos         = len(lista_nomes_autores)

    ## Monta a lista de autores com a divisão da string de nomes acima por: sobrenome, nomes
    lst_autores_artigo = []
    for cada_lista_autores in lista_nomes_autores:
        lista_organizada, erros_organizar = organizar_nomes(cada_lista_autores)
        lst_autores_artigo.append(lista_organizada)
    print(f'{len(lst_autores_artigo)} listas de autores de {qte_artigos} artigos publicados no período')

In [ ]:
lista_nomes_autores[:3]

In [ ]:
lst_autores_artigo[:3]

In [ ]:
a='a b c de oliveira'
padronizar_nome(a)
iniciais_nome(a)

### Execução

In [ ]:
def gerar_dicionarios(df_prod, lista_docentes, lista_discentes, inicio = min(df_prod['Ano']), final  = max(df_prod['Ano'])):
    qte_docentes     = len(lista_docentes)
    qte_discentes    = len(lista_discentes)
    total_iteracoes  = (qte_docentes+qte_discentes)
    
    ## Monta uma lista de strings com nomes dos autores de cada artigo na forma extraída pelo e-lattes
    lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
    qte_artigos         = len(lista_nomes_autores)
        
    ## Define os limites para considerar duas strings com nomes similares entre si
    limite_jaro_nome        = 0.88
    limite_jaro_iniciais    = 0.75
    limite_jaccard_iniciais = 0.32

    ## Função para calcular similaridade dos nomes de docentes com cada nome de autor da lista de autores de cada artigo
    t1 = time.time()
    dic_nomes_docentes  = {}
    dic_nomes_discentes = {}    
    erros=[]
    rot1='Comparando nome do docente'
    rot2='Sobren/Iniciais docen'
    rot3='Sobrenome/Iniciais autor'
    rot4='I.Doc'
    rot5='I.Aut'
    rot6='Jaro-Winkler'
    rot7='Jaccard'
    for m,docente in enumerate(lista_docentes):
        achados_docentes     = []
        lista_indice_docente = []
        lista_qualis_docente = []
        docentes_naoencontrados = []          
        contagem=m+1
        # clear_output(wait=True)       
        try:
            nome_docente_padronizado = padronizar_nome(docente).lower()
            iniciais_nome_docente    = iniciais_nome(docente).lower()
            string_iniciais_docente  = ' '.join(x.strip() for x in iniciais_nome_docente.split(',')[1:])
            set_iniciais_docente     = converter_lista_set([x.strip() for x in iniciais_nome_docente.split(',')[1:]])                
            print(f'\nCálculo das similaridades autores-docentes (nome/iniciais/Jaccard):')
            print(f'{rot1:^40} |{rot2:^20}| {rot3:^25}|{rot4:^5}|{rot5:^5}|{rot6:^15}|{rot7:^10}')
            ## Monta a lista de autores com a divisão da string organizada padronizada no formato: {sobrenome, iniciais de nomes}
            lst_autores_artigo = []
            lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
            for n,nomes_autor in enumerate(lista_nomes_autores):
                autores,erros = organizar_nomes(nomes_autor)
                for o,autor in enumerate(autores):
                    qte_autores_artigo = len(autores)
                    # print(f'{o+1:2}/{qte_autores_artigo:2} nomes de autor do artigo em análise, restando {qte_autores_artigo-o-1:2}')
                    nome_autor_padronizado = padronizar_nome(autor).lower()
                    try:    
                        iniciais_nome_autor    = iniciais_nome(autor).lower()
                        string_iniciais_autor  = ' '.join(x.strip() for x in iniciais_nome_autor.split(',')[1:])
                        set_iniciais_autor     = converter_lista_set([x.strip() for x in iniciais_nome_autor.split(',')[1:]])
                        # time.sleep(1)
                        ## Cálculos de similaridades entre os nomes do autor e do docente
                        if iniciais_nome_docente and iniciais_nome_autor != '':
                            similaridade_nome     = get_jaro_distance(iniciais_nome_docente, iniciais_nome_autor)
                            similaridade_iniciais = get_jaro_distance(string_iniciais_docente, string_iniciais_autor)
                            similaridade_jaccard  = np.round(jaccard_similarity(set_iniciais_docente, set_iniciais_autor),2)
                            print(f'\nCálculo das similaridades autores-discentes (nome/iniciais/Jaccard):')
                            print(f'{padronizar_nome(docente):^40} | {iniciais_nome_docente:<20}|{iniciais_nome_autor:<20} {similaridade_nome:^5}|{string_iniciais_docente:^5}|{string_iniciais_autor:^5}|{similaridade_iniciais:^15}|{similaridade_jaccard:^10}')
                            clear_output(wait=True)
                            if similaridade_nome > limite_jaro_nome and similaridade_iniciais > limite_jaro_iniciais and similaridade_jaccard > limite_jaccard_iniciais:
                                if n not in lista_indice_docente:
                                    print(f'Similaridade encontrada no artigo {n+1}/{qte_artigos}, docente {m+1}/{qte_docentes}')
                                    print(f'{rot1:^40} |{rot2:^20}| {rot3:^25}|{rot4:^5}|{rot5:^5}|{rot6:^15}|{rot7:^10}')
                                    print(f'{padronizar_nome(docente):^40} | {iniciais_nome_docente:<20}|{iniciais_nome_autor:<20} {similaridade_nome:^5}|{string_iniciais_docente:^5}|{string_iniciais_autor:^5}|{similaridade_iniciais:^15}|{similaridade_jaccard:^10}')
                                    achados_docentes.append(docente)
                                    lista_indice_docente.append(n)
                                    # print(len(lista_indice_docente))
                                    extrato = df_prod['Qualis'][n]
                                    lista_qualis_docente.append(extrato)
                                    clear_output(wait=True)                                    
                    except Exception as e1:
                        # print(f'Erro na etapa 1 de gerar_dicionarios, ao tratar iniciais do nome de autor/docente da linha  {n}/{o}/{qte_docentes}/{qte_artigos}')
                        # print(e1)
                        similaridade_iniciais = np.NaN
                        similaridade_jaccard  = np.NaN
                        erros.append(('e1_similaridadesdocente',m,n,o,e1))                    
                lst_autores_artigo.append(iniciais_nome_autor)
            # print(f'Lista organizada de nomes de autores: {len(lst_autores_artigo)} listas de autores de artigos publicados no período')

            ## Ao final da leitura todos artigos para cada docente, criar o dicionário de docentes quando docente tenha aparecido na linha de autores
            if lista_indice_docente != []:
                dic_nomes_docentes[m] = (docente, lista_indice_docente, lista_qualis_docente)
            else:
                docentes_naoencontrados.append(docente)

            tdec=time.time()-t1
            # tres=tdec/(m+1)*((qte_docentes-m)+qte_discentes)
            # print(f'Analisadas{m+1:4}/{total_iteracoes} iterações em {horas(tdec)}, restando {total_iteracoes-m}. Busca{m+1:4}/{qte_docentes:<4}docente: {docente.title():50}')
            # print(f'Nome do docente foi encontrado em {len(lista_indice_docente):2} artigos')
        except Exception as e2:
            # print(f'Erro na etapa 2 de gerar_dicionarios, ao calcular similaridades de autor/docente  da linha {n}/{o}/{qte_docentes}/{qte_artigos}')
            # print(e2)
            erros.append(('e2_padronizardocentes',m,n,o,e2)) 

    ## Função para calcular similaridade dos nomes de discentes com cada nome de autor da lista de autores de cada artigo
    for p,discente in enumerate(lista_discentes):
        achados_discentes     = []
        lista_indice_discente = []
        lista_qualis_discente = []
        discentes_naoencontrados = []
        contagem=m+p+1
        # clear_output(wait=True)       
        try:
            nome_discente_padronizado = padronizar_nome(discente).lower()
            iniciais_nome_discente    = iniciais_nome(discente).lower()
            string_iniciais_discente  = ' '.join(x.strip() for x in iniciais_nome_discente.split(',')[1:])
            set_iniciais_discente     = converter_lista_set([x.strip() for x in iniciais_nome_discente.split(',')[1:]])
            rot2='Sobren/Iniciais disce'
            rot4='I.Dis'            
            ## Monta a lista de autores com a divisão da string organizada padronizada no formato: {sobrenome, iniciais de nomes}
            lst_autores_artigo = []
            lista_nomes_autores = [limpar_nomes(x) for x in df_prod['AUTORES'].values]
            for n,nomes_autor in enumerate(lista_nomes_autores):
                # clear_output(wait=True)
                # print(f'Procurando {nome_discente_padronizado.title()} em {n+1:2}/{qte_artigos:<2} artigos, restando {qte_artigos-n-1:<2}')
                autores,erros = organizar_nomes(nomes_autor)
                for q,autor in enumerate(autores):
                    qte_autores_artigo = len(autores)
                    nome_autor_padronizado = padronizar_nome(autor).lower()
                    # print(f'{q+1:2}/{qte_autores_artigo:2} nomes de autor do artigo em análise, restando {qte_autores_artigo-q-1:2}')
                    nome_autor_padronizado = padronizar_nome(autor).lower()
                    try:    
                        iniciais_nome_autor    = iniciais_nome(autor).lower()
                        string_iniciais_autor  = ' '.join(x.strip() for x in iniciais_nome_autor.split(',')[1:])
                        set_iniciais_autor     = converter_lista_set([x.strip() for x in iniciais_nome_autor.split(',')[1:]])   
                        ## Cálculos de similaridades entre os nomes do autor e do discente
                        if iniciais_nome_discente and iniciais_nome_autor != '':
                            similaridade_nome     = get_jaro_distance(iniciais_nome_discente, iniciais_nome_autor)
                            similaridade_iniciais = get_jaro_distance(string_iniciais_discente, string_iniciais_autor)
                            similaridade_jaccard  = np.round(jaccard_similarity(set_iniciais_discente, set_iniciais_autor),2)
                            print(f'{padronizar_nome(discente):^40} | {iniciais_nome_discente:<20}|{iniciais_nome_autor:<20} {similaridade_nome:^5}|{string_iniciais_docente:^5}|{string_iniciais_autor:^5}|{similaridade_iniciais:^15}|{similaridade_jaccard:^10}')                            
                            if similaridade_nome > limite_jaro_nome and similaridade_iniciais > limite_jaro_iniciais and similaridade_jaccard > limite_jaccard_iniciais:
                                if n not in lista_indice_discente:
                                    print(f'Similaridade encontrada no artigo {n+1}/{qte_artigos}, discente {p+1}/{qte_discentes}')
                                    print(f'{rot1:^40} |{rot2:^20}| {rot3:^25}|{rot4:^5}|{rot5:^5}|{rot6:^15}|{rot7:^10}')
                                    print(f'{padronizar_nome(discente):^40} | {iniciais_nome_discente:<20}|{iniciais_nome_autor:<20} {similaridade_nome:^5}|{string_iniciais_discente:^5}|{string_iniciais_autor:^5}|{similaridade_iniciais:^15}|{similaridade_jaccard:^10}')
                                    achados_discentes.append(discente)
                                    lista_indice_discente.append(n)
                                    # print(len(lista_indice_discente))
                                    extrato = df_prod['Qualis'][n]
                                    lista_qualis_discente.append(extrato)
                                    clear_output(wait=True)
                    except Exception as e3:
                        similaridade_iniciais = np.NaN
                        similaridade_jaccard  = np.NaN
                        # print(f'Erro na etapa 3 de gerar_dicionarios, ao calcular similaridades de nomes de autor/discente da linha {n}/{o}/{qte_discentes}/{qte_artigos}')
                        # print(e3)
                        erros.append(('e3_buscadiscentes',p,n,q,e3))
        
            ## Ao final da leitura todos artigos para cada discente, criar o dicionário de discente quando docente tenha aparecido na linha de autores
            if lista_indice_discente != []:
                dic_nomes_discentes[o] = (discente, lista_indice_discente, lista_qualis_discente)
            else:
                discentes_naoencontrados.append(discente)            
            
            tdec=time.time()-t1
            # tres=tdec/(o+p+1)*((qte_discentes-p)+qte_discentes)
            # print(f'Analisadas{contagem+1:4}/{total_iteracoes} iterações em {horas(tdec)}, restando {horas(tres)} para iterar {total_iteracoes-contagem-1}. Busca{p+1:4}/{qte_discentes:<4}discente: {discente.title():50}')
            # print(f'Nome do discente foi encontrado em {len(lista_indice_discente):2} artigos')   
        except Exception as e4:
            # print('Erro na etapa 4 de gerar_dicionarios, ao finalizar montagem dos dicionários:',e4)
            erros.append(('e4_padronizardiscente',m,n,e4))

    return dic_nomes_docentes, dic_nomes_discentes, docentes_naoencontrados, discentes_naoencontrados, erros

In [ ]:
dic_nomes_docentes, dic_nomes_discentes, docentes_naoencontrados, discentes_naoencontrados, erros = gerar_dicionarios(df_prod, lista_docentes, lista_discentes)

In [ ]:
# dic_nomes_docentes

In [ ]:
print(len(docentes_naoencontrados),'total de nomes de  docentes não encontrados nos artigos')
print(len(discentes_naoencontrados),'total de nomes de discentes não encontrados nos artigos')

In [ ]:
discentes_naoencontrados

In [ ]:
len(erros)

In [ ]:
erros[:3]

In [ ]:
# lista_artigos_problemas = []
# lista_autores_problemas = []
# for etapa,docente,i,discente,artigo in erros:
#     if i not in lista_artigos_problemas:
#         lista_artigos_problemas.append(i)
#         lista_autores_problemas.append(df_prod['AUTORES'][i])

# df_artigos_problemas=pd.DataFrame(lista_artigos_problemas).reset_index(drop=True)
# df_artigos_problemas['LISTA_AUTORES'] = lista_autores_problemas
# df_artigos_problemas

In [ ]:
# ## Lista ordenada alfabeticamente pelos sobrenomes de docentes
# lista_docentes_sobrenome=[]
# for i in lista_docentes:
#     lista_docentes_sobrenome.append(iniciais_nome(i))
    
# lista_docentes_sobrenome.sort()
# for j in lista_docentes_sobrenome:
#     print(f'{j.lower()}')

In [ ]:
inicio = 2017
final  = 2022
df_impacto_docente, df_impacto_discente = montardf_impacto_docente_discente(df_prod, dic_nomes_docentes, dic_nomes_discentes, inicio, final)
df_impacto_docente

In [ ]:
df_docentes = montardf_docentes(lista_nomes_docentes_permanentes, lista_nomes_docentes_colaboradores)
df_docentes_pcd_impacto, apuracao_pcd, df_abaixo_pcd, df_atingiram_pcd, indicador_pcd, indicador_impacto = apurar_pcd_impacto(df_docentes, df_impacto_docente, df_impacto_discente, meta_pcd=50.0, meta_impacto=150.0)

In [ ]:
plotar_pcd(df_docentes_pcd_impacto)
plotar_medias_impacto(df_docentes_pcd_impacto, grupo=False, inicio=False, final=False)

# Apuração segmentada por períodos e grupos

In [ ]:
evolucao_pcd=[]
evolucao_impacto=[]
tipo_analise=[]
grupo_analise=[]
periodo_analise=[]

## Quadriênio 2017-2020 de Docentes Permanentes

In [ ]:
lista_nomes_docentes_permanentes   = 'lista_docentes_permanentes.csv'
lista_nomes_docentes_colaboradores = 'lista_docentes_colaboradores.csv'
df_docentes = montardf_docentes(lista_nomes_docentes_permanentes, lista_nomes_docentes_colaboradores)

In [ ]:
inicio = 2017
final  = 2020
tipo   = 'artigos'
grupo  = 'permanentes'
lista_nomes_docentes = 'lista_docentes_permanentes.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)
evolucao_pcd.append(indicador_pcd)
evolucao_impacto.append(indicador_impacto)
tipo_analise.append(tipo)
grupo_analise.append(grupo)
periodo_analise.append([inicio,final])

## .
## Quadriênio 2017-2020 de Docentes Colaboradores

In [ ]:
inicio = 2017
final  = 2020
tipo   = 'artigos'
grupo  = 'colaboradores'
lista_nomes_docentes = 'lista_docentes_colaboradores.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)
evolucao_pcd.append(indicador_pcd)
evolucao_impacto.append(indicador_impacto)
tipo_analise.append(tipo)
grupo_analise.append(grupo)
periodo_analise.append([inicio,final])

# .
# .
# .
# Avaliação de meio termo biênio [2021-2022]

## Biênio 2021-2022 de Docentes Permanentes

In [ ]:
inicio = 2021
final  = 2022
tipo   = 'artigos'
grupo  = 'permanentes'
lista_nomes_docentes = 'lista_docentes_permanentes.csv'
# lista_nomes_docentes = 'lista_docentes_colaboradores.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)
evolucao_pcd.append(indicador_pcd)
evolucao_impacto.append(indicador_impacto)
tipo_analise.append(tipo)
grupo_analise.append(grupo)
periodo_analise.append([inicio,final])

## .
## .
## Biênio 2021-2022 de Docentes Colaboradores

In [ ]:
inicio = 2021
final  = 2022
tipo   = 'artigos'
grupo  = 'colaboradores'
# lista_nomes_docentes = 'lista_docentes_permanentes.csv'
lista_nomes_docentes = 'lista_docentes_colaboradores.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)
evolucao_pcd.append(indicador_pcd)
evolucao_impacto.append(indicador_impacto)
tipo_analise.append(tipo)
grupo_analise.append(grupo)
periodo_analise.append([inicio,final])

# Evolução de indicadores de gestão do programa

In [ ]:
df_indicadores = pd.DataFrame({
    'TIPO': pd.Series(tipo_analise),
    'GRUPO': pd.Series(grupo_analise),
    'PERIODOS': pd.Series(periodo_analise),
    'META_PCD_50%': pd.Series(evolucao_pcd),
    'META_IMPACTO_150ANO': pd.Series(evolucao_impacto),
    })

In [ ]:
df_indicadores.sort_values(by=['GRUPO'], ascending=True).reset_index(drop=True)

# Conferência em detalhes

In [ ]:
inicio = 2021
final  = 2022
tipo   = 'artigos'
grupo  = 'permanentes'
lista_nomes_docentes = 'lista_docentes_permanentes.csv'
# lista_nomes_docentes = 'lista_docentes_colaboradores.csv'

df_public, df_impacto_docente, df_impacto_discente, df_docentespcd, indicador_pcd, indicador_impacto = avaliar_completo(inicio, final, tipo, grupo, lista_nomes_docentes)

In [ ]:
# df_impacto_docente[:60]

## Conferência dos achados de nomes de discentes

In [ ]:
docente = 'Olindo Assis Martins Filho'
listar_achados(docente)

In [ ]:
# def separar_iniciais(nome):
#     import re
#     letras_duasconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2}$')            # Duas Letras consoantes maiúsculas juntas do início ao final da string
#     letras_tresconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3}$')            # Três Letras consoantes maiúsculas juntas do início ao final da string
#     letras_duasconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2},$')       # Duas Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
#     letras_tresconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3},$')       # Três Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
#     partes_nome=[]
#     for j in nome.split(' '):
#         div_ltrduasconsnts = letras_duasconsnts.findall(j)
#         div_ltrtriplicadas = letras_tresconsnts.findall(j)
#         div_ltrduasconsntsvirg = letras_duasconsntsvirg.findall(j)
#         div_ltrtresconsntsvirg = letras_tresconsntsvirg.findall(j)
#         if div_ltrduasconsnts or div_ltrtriplicadas:
#             iniciais_separadas = ' '.join(x for x in j)
#             partes_nome.append(iniciais_separadas)
#         elif div_ltrduasconsntsvirg or div_ltrtresconsntsvirg:
#             iniciais_separadas = ' '.join(x for x in j[:-1])
#             partes_nome.append(iniciais_separadas+',')
#         else:
#             partes_nome.append(j)
#     nome_separado = ' '.join(x for x in partes_nome).strip()
#     return nome_separado

# Outras funcionalidades

    Formatar tempo (h:min:seg)

In [ ]:
def tempo(start, end):
    t=end-start

    tempo = timedelta(
        weeks   = t//(3600*24*7),
        days    = t//(3600*24),
        seconds = t,
        minutes = t//(60),
        hours   = t//(3600),
        microseconds=t//1000000,
        )
    fmt='{H:2}:{M:02}:{S:02}'
    return strfdelta(tempo)


def horas(segundos): 
    return time.strftime("%H:%M:%S", time.gmtime(segundos)) 


def dias_horas_minutos(td):
    x = (td.days, td.seconds//3600, (td.seconds//60)%60, td.seconds)
    return x #(days, hrs, mins, seconds)


def strfdelta(tdelta, fmt='{H:02}h {M:02}m {S:02}s', inputtype='timedelta'):
    """Convert a datetime.timedelta object or a regular number to a custom-formatted string, 
    just like the stftime() method does for datetime.datetime objects.

    The fmt argument allows custom formatting to be specified.  Fields can 
    include seconds, minutes, hours, days, and weeks.  Each field is optional.

    Some examples:
        '{D:02}d {H:02}h {M:02}m {S:02}s' --> '05d 08h 04m 02s' (default)
        '{W}w {D}d {H}:{M:02}:{S:02}'     --> '4w 5d 8:04:02'
        '{D:2}d {H:2}:{M:02}:{S:02}'      --> ' 5d  8:04:02'
        '{H}h {S}s'                       --> '72h 800s'

    The inputtype argument allows tdelta to be a regular number instead of the  
    default, which is a datetime.timedelta object.  Valid inputtype strings: 
        's', 'seconds', 
        'm', 'minutes', 
        'h', 'hours', 
        'd', 'days', 
        'w', 'weeks'
    """

    # Convert tdelta to integer seconds.
    if inputtype == 'timedelta':
        remainder = int(tdelta.total_seconds())
    elif inputtype in ['s', 'seconds']:
        remainder = int(tdelta)
    elif inputtype in ['m', 'minutes']:
        remainder = int(tdelta)*60
    elif inputtype in ['h', 'hours']:
        remainder = int(tdelta)*3600
    elif inputtype in ['d', 'days']:
        remainder = int(tdelta)*86400
    elif inputtype in ['w', 'weeks']:
        remainder = int(tdelta)*604800

    f = Formatter()
    desired_fields = [field_tuple[1] for field_tuple in f.parse(fmt)]
    possible_fields = ('W', 'D', 'H', 'M', 'S')
    constants = {'W': 604800, 'D': 86400, 'H': 3600, 'M': 60, 'S': 1}
    values = {}
    
    for field in possible_fields:
        if field in desired_fields and field in constants:
            values[field], remainder = divmod(remainder, constants[field])
    
    return f.format(fmt, **values)


# print (timedelta(days=365, hours=8, minutes=15))
# print ("   Hoje é: " + str(date.today()))
# print ("Agora são: " + str(datetime.now()))
# print ("Um ano no futuro estaremos em:" + str(dt.today() + timedelta(days=365)))
# hoje = date.today()
# print(hoje)
# hora = dt.now()
# print(hora)
# dias_ano = date(hoje.year, 1, 1)
# if dias_ano < hoje:
#     print ("Decoridos %d dias do ano" % ((hoje - dias_ano).days))
    
# from datetime import datetime
# now= datetime.now() #get the current date and time

# #%c - local date and time, %x-local's date, %X- local's time
# print(now.strftime("%c"))
# print(now.strftime("%x"))
# print(now.strftime("%X"))

# ##### Time Formatting ####
# #%I/%H - 12/24 Hour, %M - minute, %S - second, %p - local's AM/PM
# print(now.strftime("%I:%M:%S %p")) # 12-Hour:Minute:Second:AM
# print(now.strftime("%H:%M")) # 24-Hour:Minute

    Padronizar nomes de autores

In [ ]:
def limpar_nomes(linha_texto):
    '''
    Retira erros e sujeira da string formada pela lista de nomes de autor dos artigos retirada do Lattes
     Recebe: Uma string com os nomes de autor
    Retorna: Uma string com os nomes de autor, removidas preposições, acentos, e demais erros pontuais
    '''
    import unicodedata
    import re
    # print('               Analisando:',linha_texto)
    string = linha_texto.replace('Network for Genomic Surveillance in South Africa;Network for Genomic Surveillance in South Africa (NGS-SA);10.1002/jmv.27190;','')
    string = string.replace('Autores: ','').replace('(Org)','').replace('(Org.)','').replace('et. al.','').replace('et al','').replace('(Org).','').replace('.','').replace('\'','')
    string = string.replace(',,,',',').replace(',,',',').replace(';',', ').replace('-',' ').replace('S?', 'SA').replace('S?', 'SA').replace('ARA?JO', 'ARAUJO').replace('FL?VIO','FLAVIO').replace('F?BIO','FABIO').replace('VIT?RIO','VITORIO')
    string = re.sub(r'[0-9]+', '', string)
    partes_string = string.split(' ')

    ## Retirar partes de nomes caso sejam preposições
    preposicoes = ['da', 'de', 'do', 'das', 'dos', ' e ']
    string = ' '.join(x for x in partes_string if x.lower() not in preposicoes)

    ## Retirar símbolos não unicode, como acentuação gráfica e cedilha
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', string) if not unicodedata.combining(ch))
    
    ## Retirar iniciais juntas, separando-as com espaço em branco
    letras_duasconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2}$')            # Duas Letras consoantes maiúsculas juntas do início ao final da string
    letras_tresconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3}$')            # Três Letras consoantes maiúsculas juntas do início ao final da string
    letras_duasconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2},$')       # Duas Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
    letras_tresconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3},$')       # Três Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
    
    partes_nome=[]
    for j in string.split(' '):
        div_ltrduasconsnts = letras_duasconsnts.findall(j)
        div_ltrtriplicadas = letras_tresconsnts.findall(j)
        div_ltrduasconsntsvirg = letras_duasconsntsvirg.findall(j)
        div_ltrtresconsntsvirg = letras_tresconsntsvirg.findall(j)
        if div_ltrduasconsnts or div_ltrtriplicadas:
            iniciais_separadas = ' '.join(x for x in j)
            partes_nome.append(iniciais_separadas)
        elif div_ltrduasconsntsvirg or div_ltrtresconsntsvirg:
            iniciais_separadas = ' '.join(x for x in j[:-1])
            partes_nome.append(iniciais_separadas+',')
        else:
            partes_nome.append(j)
    string = ' '.join(x for x in partes_nome).strip()
    return string



def padronizar_nome(linha_texto):
    '''
    Procura sobrenomes e abreviaturas e monta nome completo
     Recebe: String com todos os sobrenomes e nomes, abreviados ou não
    Retorna: Nome completo no formato padronizado em SOBRENOME AGNOME, Partes de nomes
      Autor: Marcos Aires (Mar.2022)
    '''
    import unicodedata
    import re
    # print('               Analisando:',linha_texto)
    partes_string = linha_texto.split(' ')

    ## Retirar partes de nomes caso sejam preposições
    preposicoes = ['da', 'de', 'do', 'das', 'dos', ' e ']
    string = ' '.join(x for x in partes_string if x.lower() not in preposicoes)

    ## Retirar símbolos não unicode, como acentuação gráfica e cedilha
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', string) if not unicodedata.combining(ch))
    
    ## Expressões regulares para encontrar padrões de divisão de nomes de autores
    sobrenome_inicio   = re.compile(r'^[A-ZÀ-ú-a-z]+,')                  # Sequência de letras maiúsculas no início da string
    sobrenome_composto = re.compile(r'^[A-ZÀ-ú-a-z]+[ ][A-ZÀ-ú-a-z]+,')  # Duas sequências de letras no início da string, separadas por espaço, seguidas por vírgula
    letra_abrevponto   = re.compile(r'^[A-Z][.]')                        # Uma letra maiúscula no início da string, seguida por ponto
    letra_abrevespaco  = re.compile(r'^[A-Z][ ]')                        # Uma letra maiúscula no início da string, seguida por espaço
    letras_dobradas    = re.compile(r'[A-Z]{2}')                         # Duas letras maiúsculas juntas
    letras_dobradasini = re.compile(r'[A-Z]{2}[ ]')                      # Duas letras maiúsculas juntas, seguidas por espaço
    letras_dobradasfim = re.compile(r'[ ][A-Z]{2}')                      # Duas letras maiúsculas juntas, precedidas por espaço

    ## Expressões regulares para encontrar iniciais juntas, separando-as com espaço em branco
    letras_duasconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2}$')            # Duas Letras consoantes maiúsculas juntas do início ao final da string
    letras_tresconsnts = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3}$')            # Três Letras consoantes maiúsculas juntas do início ao final da string
    letras_duasconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{2},$')       # Duas Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula
    letras_tresconsntsvirg = re.compile(r'^[B-DF-HJ-NP-TV-XZ-r]{3},$')       # Três Letras consoantes maiúsculas juntas do início ao final da string, seguidas de vírgula

    ## Agnomes e preprosições a tratar, agnomes vão em maiúsculas para sobrenome e preposições vão em minúsculas para restante das partes de nomes
    nomes=[]
    agnomes       = ['NETO','JUNIOR','FILHO','SEGUNDO','TERCEIRO','SOBRINHO']
    preposicoes   = ['de','da','do','das','dos', ' e ']
    nome_completo = ''
    
    ## Ajustar lista de termos, identificar sobrenomes compostos e ajustar sobrenome com ou sem presença de vírgula
    div_sobrenome   = sobrenome_inicio.findall(string)
    div_sbrcomposto = sobrenome_composto.findall(string)
    
    # print('-'*100)
    # print('                 Recebido:',string)
    
    ## Caso haja vírgulas na string, tratar sobrenomes e sobrenomes compostos
    if div_sobrenome != [] or div_sbrcomposto != []:
        # print('CASO_01: Há víruglas na string')
        div = string.split(', ')
        sobrenome     = div[0].strip().upper()
        try:
            div_espaco    = div[1].split(' ')
        except:
            div_espaco    = ['']
        primeiro      = div_espaco[0].strip('.').strip()
        
        # print('     Dividir por vírgulas:',div)
        # print('      Primeira DivVirgula:',sobrenome)
        # print('Segunda DivVrg/DivEspaços:',div_espaco)
        # print('      Primeira DivEspaços:',primeiro)
               
        # Caso primeiro nome seja somente duas letras maiúsculas juntas, trata-se de duas iniciais
        if len(primeiro)==2 or letras_tresconsnts.findall(primeiro) or letras_duasconsnts.findall(primeiro):
            # print('CASO_01.a: Há duas letras ou três letras consoantes juntas, são iniciais')
            primeiro_nome=primeiro[0].strip()
            # print('          C01.a1_PrimNome:',primeiro_nome)
            nomes.append(primeiro[1].strip().upper())
            try:
                nomes.append(primeiro[2].strip().upper())
            except:
                pass
            try:
                nomes.append(primeiro[3].strip().upper())
            except:
                pass            
        else:
            # print('CASO_01.b: Primeiro nome maior que 2 caracteres')
            primeiro_nome = div_espaco[0].strip().title()
            # print('          C01.a2_PrimNome:',primeiro_nome)
        
        ## Montagem da lista de partes de nomes do meio
        for nome in div_espaco:
            # print('CASO_01.c: Para cada parte de nome da divisão por espaços após divisão por vírgula')
            if nome not in nomes and nome.lower()!=primeiro_nome.lower() and nome.lower() not in primeiro_nome.lower() and nome!=sobrenome:   
                # print('CASO_01.c1: Se o nome não está nem como primeiro nome, nem sobrenomes')
                # print(nome, len(nome))
                
                ## Avaliar se é abreviatura seguida de ponto e remover o ponto
                if len(nome)<=2 and nome.lower() not in preposicoes:
                    # print('    C01.c1.1_Nome<=02:',nome)
                    for inicial in nome:
                        # print(inicial)
                        if inicial not in nomes and inicial not in primeiro_nome:
                            nomes.append(inicial.replace('.','').strip().title())
                elif len(nome)==3 and nome.lower() not in preposicoes:
                        # print('    C01.c1.2_Nome==03:',nome)
                        for inicial in nome:
                            if inicial not in nomes and inicial not in primeiro_nome:
                                nomes.append(inicial.replace('.','').strip().title())
                else:
                    if nome not in nomes and nome!=primeiro_nome and nome!=sobrenome and nome!='':
                        if nome.lower() in preposicoes:
                            nomes.append(nome.replace('.','').strip().lower())
                        else:
                            nomes.append(nome.replace('.','').strip().title())
                        # print(nome,'|',primeiro_nome)
                        
        ## Caso haja sobrenome composto que não esteja nos agnomes considerar somente primeira parte como sobrenome
        if div_sbrcomposto !=[] and sobrenome.split(' ')[1] not in agnomes and sobrenome.split(' ')[0].lower() not in preposicoes:
            # print('CASO_01.d: Sobrenome composto sem agnomes')
            # print(div_sbrcomposto)
            # print('Sobrenome composto:',sobrenome)
            
            nomes.append(sobrenome.split(' ')[1].title())
            sobrenome = sobrenome.split(' ')[0].upper().strip()
            # print('Sobrenome:',sobrenome)
            
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('    Nomes:',nomes)
        
        ## Caso haja preposição como agnome desconsiderar e passar para final dos nomes
        if div_sbrcomposto !=[] and sobrenome.split(' ')[0].lower() in preposicoes:
            # print('CASO_01.e: Preposição no Sobrenome passar para o final dos nomes')
            # print('   div_sbrcomposto:', div_sbrcomposto)
            # print('Sobrenome composto:',div_sbrcomposto)
            
            nomes.append(div_sbrcomposto[0].split(' ')[0].lower())
            # print('    Nomes:',nomes)
            sobrenome = div_sbrcomposto[0].split(' ')[1].upper().strip(',').strip()
            # print('Sobrenome:',sobrenome)
            
            for i in nomes:
                # print('CASO_01.e1: Para cada nome avaliar se o sobrenome está na lista')
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('  Nomes:',nomes)
        
        # print('Ao final do Caso 01')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('           Lista de nomes:',nomes, len(nomes),'nomes')
        
    ## Caso não haja vírgulas na string considera sobrenome o último nome da string dividida com espaço vazio
    else:
        # print('CASO_02: Não há víruglas na string')
        try:
            div = string.split(' ')
            # print('      Divisões por espaço:',div)
            
            if div[-1] in agnomes: # nome final é um agnome
                sobrenome     = div[-2].upper().strip()+' '+div[-1].upper().strip()
                for i in div[1:-2]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.title().strip())
                    if i in preposicoes:
                        nomes.append(i.lower().strip())
            else:
                if len(div[-1]) > 2:
                    sobrenome     = div[-1].upper().strip()
                    primeiro_nome = div[1].title().strip()
                    for i in div[1:-1]:
                        if i != sobrenome and i not in preposicoes:
                            nomes.append(i.title().strip())
                        if i in preposicoes:
                            nomes.append(i.lower().strip())
                else:
                    sobrenome     = div[-2].upper().strip()
                    for i in div[-1]:
                        nomes.append(i.title())
                    primeiro_nome = nomes[0].title().strip()
                    for i in div[1:-1]:
                        if i != sobrenome and i not in preposicoes:
                            nomes.append(i.title().strip())
                        if i in preposicoes:
                            nomes.append(i.lower().strip())
        except:
            sobrenome = div[-1].upper().strip()
            for i in div[1:-1]:
                    if i != sobrenome and i not in preposicoes:
                        nomes.append(i.title().strip())
                    if i in preposicoes:
                        nomes.append(i.lower().strip())
            
        if sobrenome.lower() != div[0].lower().strip():
            primeiro_nome=div[0].title().strip()
        else:
            primeiro_nome=''
        
        # print('Ao final do Caso 02')
        # print('    Sobrenome sem vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome sem vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio sem vírgula:',nomes, len(nomes),'nomes')
    
    ## Encontrar e tratar como abreviaturas termos com apenas uma ou duas letras iniciais juntas, com ou sem ponto
    for j in nomes:
        # print('CASO_03: Avaliar cada nome armazenado na variável nomes')
        # Procura padrões com expressões regulares na string
        div_sobrenome      = sobrenome_inicio.findall(j)
        div_sbrcomposto    = sobrenome_composto.findall(j)
        div_abrevponto     = letra_abrevponto.findall(j)
        div_abrevespaco    = letra_abrevespaco.findall(j)
        div_ltrdobradasini = letras_dobradasini.findall(j)
        div_ltrdobradasfim = letras_dobradasfim.findall(j)
        div_ltrdobradas    = letras_dobradas.findall(j)
        div_ltrduasconsnts = letras_duasconsnts.findall(j)
        div_ltrtriplicadas = letras_tresconsnts.findall(j)
        tamanho=len(j)
        # print('\n', div_ltrdobradasini, div_ltrdobradasfim, tamanho, 'em:',j,len(j))
        
        ## Caso houver abreviatura com uma letra em maiúscula nos nomes
        if div_abrevponto !=[] or tamanho==1:
            # print('CASO_03.1: Há abreviaturas uma letra maiúscula nos nomes')
            nome = j.replace('.','').strip()
            if nome not in nomes and nome != sobrenome and nome != primeiro_nome:
                # print('CASO_03.1a: Há abreviaturas uma letra maiúscula nos nomes')
                nomes.append(nome.upper())
        
        ## Caso houver duas inicias juntas em maiúsculas
        elif div_ltrdobradasini !=[] or div_ltrdobradasfim !=[] or div_ltrdobradas !=[] :
            # print('CASO_03.2: Há abreviaturas uma letra maiúscula nos nomes')
            for letra in j:
                # print('CASO_03.2a: Avaliar cada inicial do nome')
                if letra not in nomes and letra != sobrenome and letra != primeiro_nome:
                    # print('CASO_03.2a.1: Se não estiver adicionar inicial aos nomes')
                    nomes.append(letra.upper())
        
        # Caso haja agnomes ao sobrenome
        elif sobrenome in agnomes:
            # print('CASO_03.3: Há agnomes nos sobrenomes')
            sobrenome = nomes[-1].upper()+' '+sobrenome
            # print(sobrenome.split(' '))
            # print('Sobrenome composto:',sobrenome)
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
            
        else:
            # print('CASO_03.4: Não há agnomes nos sobrenomes')
            if j not in nomes and j not in sobrenome and j != primeiro_nome:
                if len(nomes) == 1:
                    nomes.append(j.upper())
                elif 1 < len(nomes) <= 3:
                    nomes.append(j.lower())
                else:
                    nomes.append(j.title())
         
        # print('Ao final do Caso 03')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
        
    nomes_meio=' '.join([str for str in nomes]).strip()
    # print('        Qte nomes do meio:',nomes,len(nomes))
    
    if primeiro_nome.lower() == sobrenome.lower():
        # print('CASO_04: Primeiro nome é igual ao sobrenome')
        try:
            primeiro_nome=nomes_meio.split(' ')[0]
        except:
            pass
        try:
            nomes_meio.remove(sobrenome)
        except:
            pass
    
        # print('Ao final do caso 04')
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
    
    ## Caso sobrenome seja só de 1 letra passá-lo para nomes e considerar o próximo nome como sobrenome
    for i in range(len(div)):
        if len(sobrenome)==1 or sobrenome.lower() in preposicoes:
            # print('CASO_05: Mudar sobrenomes até o adequado')
            div    = string.split(', ')
            # print('Divisão por vírgulas:',div)
            avaliar0       = div[0].split(' ')[0].strip()
            if 1< len(avaliar0) < 3:
                # print('CASO_05.1: 1 < Sobrenome < 3 fica em minúsculas')
                sbrn0          = avaliar0.lower()
            else:
                # print('CASO_05.2: Sobrenome de tamanho 1 ou maior que 3 fica em maiúsculas')
                sbrn0          = avaliar0.title()
            # print('sbrn0:',sbrn0, len(sbrn0))
            
            try:
                avaliar1=div[0].split(' ')[1].strip()
                # print('avaliar0',avaliar0)
                # print('avaliar1',avaliar1)
                if 1 < len(avaliar1) <=3:
                    sbrn1     = avaliar1.lower()
                else:
                    sbrn1     = avaliar1.title()
                # print('sbrn1:',sbrn1, len(sbrn1))

            except:
                pass

            if div != []:
                # print('CASO_05.3: Caso haja divisão por vírgulas na string')
                try:
                    div_espaco     = div[1].split(' ')
                except:
                    div_espaco     = div[0].split(' ')
                sobrenome      = div_espaco[0].strip().upper()
                try:
                    primeiro_nome  = div_espaco[1].title().strip()
                except:
                    primeiro_nome  = div_espaco[0].title().strip()
                if len(sbrn0) == 1:
                    # print('CASO_05.3a: Avalia primeiro sobrenome de tamanho 1')
                    # print('Vai pros nomes:',str(sbrn0).title())
                    nomes_meio = nomes_meio+str(' '+sbrn0.title())
                    # print('   NomesMeio:',nomes_meio)

                elif 1 < len(sbrn0) <= 3:
                    # print('CASO_05.3b: Avalia primeiro sobrenome 1< tamanho <=3')
                    # print('Vão pros nomes sbrn0:',sbrn0, 'e sbrn1:',sbrn1)

                    div_tresconsoantes = letras_tresconsnts.findall(sobrenome)
                    if div_tresconsoantes != []:
                        # print('CASO_05.4: Três consoantes como sobrenome')
                        for letra in sobrenome:
                            nomes.append(letra)

                        if len(sobrenome) >2:
                            sobrenome=nomes[0]
                        else:
                            sobrenome=nomes[1]
                        nomes.remove(sobrenome)
                        primeiro_nome=nomes[0]
                        nomes_meio=' '.join([str for str in nomes[1:]]).strip()
                        nome_completo=sobrenome.upper()+', '+nomes_meio                
                    
                    try:                       
                        # print(' 05.3b    Lista de Nomes:',nomes_meio)
                        nomes_meio=nomes_meio.replace(sbrn0,'')
                        # print(' 05.3b ReplaceSobrenome0:',nomes_meio)
                        nomes_meio=nomes_meio.replace(sbrn1,'')
                        # print(' 05.3b ReplaceSobrenome1:',nomes_meio)
                    except Exception as e:
                        # print('   Erro ReplaceSobrenome:',e)
                        pass
                    try:
                        nomes_meio.replace(primeiro_nome.title(),'')
                        nomes_meio.replace(primeiro_nome.lower(),'')
                        nomes_meio.replace(primeiro_nome,'')
                        # print(' 05.3b Replace PrimNome:',nomes_meio)
                    except Exception as e:
                        print('Erro no try PrimeiroNome:',e)
                        pass
                    nomes_meio = nomes_meio.replace(sobrenome,'')
                    try:
                        for n,i in enumerate(avaliar1):
                            nomes.append(i.upper())
                            sbrn1     = avaliar1[0]
                        else:
                            sbrn1     = avaliar1.title()
                        # print('sbrn1:',sbrn1, len(sbrn1))
                        nomes_meio = nomes_meio+str(' '+sbrn0)+str(' '+sbrn1)
                    except:
                        nomes_meio = nomes_meio+str(' '+sbrn0)
                    nomes      = nomes_meio.strip().strip(',').split(' ')
                    # print(' 05.3b NomesMeio:',nomes_meio)
                    # print(' 05.3b     Nomes:',nome)

                else:
                    # print('CASO_05.3c: Avalia primeiro sobrenome >3')
                    nomes_meio = nomes_meio+str(' '+div[0].strip().title())
                    nomes      = nomes_meio.strip().split(' ')
                    # print(' 05.3c NomesMeio:',nomes_meio)
                    # print(' 05.3c     Nomes:',nomes)

                nomes_meio=nomes_meio.replace(sobrenome,'').replace(',','').strip()
                nomes_meio=nomes_meio.replace(primeiro_nome,'').strip()

            # print('Ao final do caso 05')
            # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
            # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
            # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
    
    if sobrenome != '' and primeiro_nome !='':
        nome_completo=sobrenome.upper().replace(',','')+', '+primeiro_nome.replace(',','')+' '+nomes_meio.replace(sobrenome,'').replace(',','')
    elif sobrenome != '':
        nome_completo=sobrenome.upper().replace(',','')+', '+nomes_meio.replace(sobrenome,'').replace(',','')
    else:
        nome_completo=sobrenome.upper()
    
#     print('Após ajustes finais')
#     print('     Sobrenome:',sobrenome)
#     print(' Primeiro Nome:',primeiro_nome)
#     print('         Nomes:',nomes)
#     print('     NomesMeio:',nomes_meio)        
        
#     print('                Resultado:',nome_completo)
    
    return nome_completo.strip()



def padronizar_titulo(titulo_bruto):
    '''
    Retira acentos, expressão (Org.) e espaços vazios do título da publicação
    Autor: Marcos Aires (Fev.2022)
    '''
    import re
    import unicodedata
    
    ## Retirar caracteres não unicode
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', titulo_bruto) if not unicodedata.combining(ch))
    string = string.replace('(Org)','').replace('(Org.)','').replace('(Org).','').replace('.',' ').replace('-',' ').replace('\'',' ').lower()
    
    substitui_iniciais=[('rl,', 'r l,'), ('hs,', 'h s,'), ('gc,', 'g c,'), ('sf,', 's f,'), ('fo,', 'f o,'), (' oa,', ' o a,'), ('mss,', 'm s s,'), 
                        ('lagares, ma', 'lagares, m a'), ('cota, gf', 'cota, g f'), ('cota gf,', 'cota, g f,'), ('diotaiut,', 'diotaiuti,'), ('grenfell, r f q','queiroz, rafaella fortini grenfell')]
    for i in substitui_iniciais:
        string = string.replace(i[0], i[1])
    
    ## Retirar preposições
    preposicoes = ['da', 'de', 'do', 'das', 'dos']
    partes_string = string.split(' ')
    string = ' '.join(x for x in partes_string if x.lower() not in preposicoes)
    
    titulo_padronizado = string.strip().strip('"')
    
    return titulo_padronizado



def iniciais_nome(linha_texto):
    '''
    Função para retornar sobrenome+iniciais das partes de nome, na forma: SOBRENOME, X Y Z
     Recebe: String com nome
    Retorna: Tupla com nome e sua versão padronizada em sobrenome+agnome em maiúsculas, seguida de vírgula e das iniciais das demais partes de nome
      Autor: Marcos Aires (Mar.2022)
    '''
    import unicodedata
    import re
    # print('               Analisando:',linha_texto)
    
    ## Retirar caracteres não unicode
    string = ''.join(ch for ch in unicodedata.normalize('NFKD', linha_texto) if not unicodedata.combining(ch))
    string = string.replace('(Org)','').replace('(Org.)','').replace('(Org).','').replace('.','')
    
    ## Retirar preposições
    preposicoes   = ['da','de','do','das','dos']
    partes_string = string.split(' ')
    string = ' '.join(x for x in partes_string if x.lower() not in preposicoes)
        
    ## Expressões regulares para encontrar padrões de divisão de nomes de autores
    sobrenome_inicio   = re.compile(r'^[A-ZÀ-ú-a-z]+,')                 # Sequência de letras maiúsculas no início da string
    sobrenome_composto = re.compile(r'^[A-ZÀ-ú-a-z]+[ ][A-ZÀ-ú-a-z]+,') # Duas sequências de letras no início da string, separadas por espaço, seguidas por vírgula
    letra_abrevponto   = re.compile(r'^[A-Z][.]')                       # Uma letra maiúscula no início da string, seguida por ponto
    letra_abrevespaco  = re.compile(r'^[A-Z][ ]')                       # Uma letra maiúscula no início da string, seguida por espaço
    letras_dobradas    = re.compile(r'[A-Z]{2}')                        # Duas letras maiúsculas juntas no início da string, seguida por espaço
    letras_dobradasini = re.compile(r'[A-Z]{2}[ ]')                     # Duas letras maiúsculas juntas no início da string, seguida por espaço
    letras_dobradasfim = re.compile(r'[ ][A-Z]{2}')                     # Duas letras maiúsculas juntas no final da string, precedida por espaço
        
    nomes=[]
    agnomes       = ['NETO','JUNIOR','FILHO','SEGUNDO','TERCEIRO', 'SOBRINHO']
    nome_completo = ''
    
    ## Ajustar lista de termos, identificar sobrenomes compostos e ajustar sobrenome com ou sem presença de vírgula
    div_sobrenome      = sobrenome_inicio.findall(string)
    div_sbrcomposto    = sobrenome_composto.findall(string)
    
    ## Caso haja vírgulas na string, tratar sobrenomes e sobrenomes compostos
    if div_sobrenome != [] or div_sbrcomposto != []:
        div   = string.split(', ')
        sobrenome     = div[0].strip().upper()
        try:
            div_espaco    = div[1].split(' ')
        except:
            div_espaco  = ['']
        primeiro      = div_espaco[0].strip('.')
        
        ## Caso primeiro nome sejam somente duas letras maiúsculas juntas, trata-se de duas iniciais
        if len(primeiro)==2:
            primeiro_nome=primeiro[0].strip()
            nomes.append(primeiro[1].strip())
        else:
            primeiro_nome = div_espaco[0].strip().title()
        
        ## Montagem da lista de nomes do meio
        for nome in div_espaco:
            if nome not in nomes and nome.lower()!=primeiro_nome.lower() and nome.lower() not in primeiro_nome.lower() and nome!=sobrenome:   
                # print(nome, len(nome))
                
                ## Avaliar se é abreviatura seguida de ponto e remover o ponto
                if len(nome)<=2 and nome.lower() not in preposicoes:
                    for inicial in nome:
                        # print(inicial)
                        if inicial not in nomes and inicial not in primeiro_nome:
                            nomes.append(inicial.replace('.','').strip().title())
                else:
                    if nome not in nomes and nome!=primeiro_nome and nome!=sobrenome and nome!='':
                        if nome.lower() in preposicoes:
                            nomes.append(nome.replace('.','').strip().lower())
                        else:
                            nomes.append(nome.replace('.','').strip().title())
                        # print(nome,'|',primeiro_nome)
                        
        ## Caso haja sobrenome composto que não esteja nos agnomes considerar somente primeira parte como sobrenome
        if div_sbrcomposto !=[] and sobrenome.split(' ')[1] not in agnomes:
            # print(div_sbrcomposto)
            # print('Sobrenome composto:',sobrenome)
            nomes.append(sobrenome.split(' ')[1].title())
            sobrenome = sobrenome.split(' ')[0].upper()
            # print('Sobrenome:',sobrenome.split(' '))
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
        
        # print('    Sobrenome com vírgula:',sobrenome, len(sobrenome),'letras')
        # print('Primeiro nome com vírgula:',primeiro_nome, len(primeiro_nome),'letras')
        # print('Nomes do meio com vírgula:',nomes, len(nomes),'nomes')
        
    ## Caso não haja vírgulas na string considera sobrenome o último nome da string dividida com espaço vazio
    else:
        try:
            div       = string.split(' ')
            if div[-2] in agnomes:
                sobrenome = div[-2].upper()+' '+div[-1].strip().upper()
                for i in nomes[1:-2]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
            else:
                sobrenome = div[-1].strip().upper()
                for i in div[1:-1]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
        except:
            sobrenome = div[-1].strip().upper()
            for i in div[1:-1]:
                    if i not in sobrenome and i not in preposicoes:
                        nomes.append(i.strip().title())
                    if i in preposicoes:
                        nomes.append(i.strip().lower())
            
        if sobrenome.lower() != div[0].strip().lower():
            primeiro_nome=div[0].strip().title()
        else:
            primeiro_nome=''
        
        # print('    Sobrenome sem vírgula:',sobrenome)
        # print('Primeiro nome sem vírgula:',primeiro_nome)
        # print('Nomes do meio sem vírgula:',nomes)
    
    # Encontrar e tratar como abreviaturas termos com apenas uma ou duas letras iniciais juntas, com ou sem ponto
    for j in nomes:
        # Procura padrões com expressões regulares na string
        div_sobrenome      = sobrenome_inicio.findall(j)
        div_sbrcomposto    = sobrenome_composto.findall(j)
        div_abrevponto     = letra_abrevponto.findall(j)
        div_abrevespaco    = letra_abrevespaco.findall(j)
        div_ltrdobradasini = letras_dobradasini.findall(j)
        div_ltrdobradasfim = letras_dobradasfim.findall(j)
        div_ltrdobradas    = letras_dobradas.findall(j)
        tamanho=len(j)
        # print('\n', div_ltrdobradasini, div_ltrdobradasfim, tamanho, 'em:',j,len(j))
        
        #caso houver abreviatura com uma letra em maiúscula nos nomes
        if div_abrevponto !=[] or tamanho==1:
            cada_nome = j.replace('.','').strip()
            if cada_nome not in nomes and cada_nome != sobrenome and nome != primeiro_nome:
                nomes.append(cada_nome)
        
        #caso houver duas inicias juntas em maiúsculas
        elif div_ltrdobradasini !=[] or div_ltrdobradasfim !=[] or div_ltrdobradas !=[] :
            for letra in j:
                if letra not in nomes and letra != sobrenome and letra != primeiro_nome:
                    nomes.append(letra)
        
        #caso haja agnomes ao sobrenome
        elif sobrenome in agnomes:
            sobrenome = nomes[-1].upper()+' '+sobrenome
            # print(sobrenome.split(' '))
            # print('Sobrenome composto:',sobrenome)
            for i in nomes:
                if i.lower() in sobrenome.lower():
                    nomes.remove(i)
            # print('Nomes do meio:',nomes)
            
        else:
            if j not in nomes and j not in sobrenome and j != primeiro_nome:
                nomes.append(j)
    
    nomes_meio=' '.join([str[0] for str in nomes]).strip()
    # print('Qte nomes do meio',len(nomes),nomes)
    if sobrenome != '' and primeiro_nome !='':
        sobrenome_iniciais = sobrenome+', '+primeiro_nome[0]+' '+nomes_meio
    elif sobrenome != '':
        sobrenome_iniciais = sobrenome
    
    return sobrenome_iniciais.strip()


## Agregar aprendizado supervisionado humano à medida que forem sendo identificados erros na situação atual
lista_extra = [
                # ('ALBUQUERQUE, Adriano B', 'ALBUQUERQUE, Adriano Bessa'),
                # ('ALBUQUERQUE, Adriano', 'ALBUQUERQUE, Adriano Bessa'),
                # ('COELHO, Andre L V', 'COELHO, Andre Luis Vasconcelos'),
                # ('DUARTE, Joao B F', 'DUARTE, Joao Batista Furlan'),
                # ('FILHO, Raimir H','HOLANDA FILHO, Raimir'),
                # ('FILHO, Raimir','HOLANDA FILHO, Raimir'),
                # ('FORMIGO, A','FORMICO, Maria Andreia Rodrigues'),
                # ('FORMICO, A','FORMICO, Maria Andreia Rodrigues'),
                # ('FURLAN, J B D', 'FURLAN, Joao Batista Duarte'),
                # ('FURTADO, Elizabeth', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Elizabeth S', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Elizabeth Sucupira','FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, M E S', 'FURTADO, Maria Elizabeth Sucupira'),
                # ('FURTADO, Vasco', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, J P', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, J V P', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, Vasco', 'FURTADO, Joao Jose Vasco Peixoto'),
                # ('FURTADO, Elizabeth','FURTADO, Maria Elizabeth Sucupira'),
                # ('HOLANDA, Raimir', 'HOLANDA FILHO, Raimir'),
                # ('LEITE, G S', 'LEITE, Gleidson Sobreira'),
                # ('PEQUENO, T H C', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio','PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio Cavalcante', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PINHEIRO, Placido R', 'PINHEIRO, Placido Rogerio'),
                # ('PINHEIRO, Vladia', 'PINHEIRO, Vladia Celia Monteiro'),
                # ('RODRIGUES, M A F', 'RODRIGUES, Maria Andreia Formico'),
                # ('RODRIGUES, Andreia', 'RODRIGUES, Maria Andreia Formico'),
                # ('JOAO, Batista F Duarte,', 'FURLAN, Joao Batista Duarte'),
                # ('MACEDO, Antonio Roberto M de', 'MACEDO, Antonio Roberto Menescal de'),
                # ('MACEDO, D V', 'MACEDO, Daniel Valente'),
                # ('MENDONCA, Nabor C', 'MENDONCA, Nabor das Chagas'),
                # ('PEQUENO, Tarcisio', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PEQUENO, Tarcisio H', 'PEQUENO, Tarcisio Haroldo Cavalcante'),
                # ('PINHEIRO, Mirian C D', 'PINHEIRO, Miriam Caliope Dantas'),
                # ('PINHEIRO, Mirian Caliope Dantas', 'PINHEIRO, Miriam Caliope Dantas'),
                # ('PINHEIRO, P G C D', 'PINHEIRO, Pedro Gabriel Caliope Dantas'),
                # ('PINHEIRO, Pedro G C', 'PINHEIRO, Pedro Gabriel Caliope Dantas'),
                # ('PINHEIRO, Placido R', 'PINHEIRO, Placido Rogerio'),
                # ('PINHEIRO, Vladia', 'PINHEIRO, Vladia Celia Monteiro'),
                # ('ROGERIO, Placido Pinheiro', 'PINHEIRO, Placido Rogerio'),
                # ('REBOUCRAS FILHO, Pedro', 'REBOUCAS FILHO, Pedro Pedrosa'),
                # ('SAMPAIO, A', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SAMPAIO, Americo', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SAMPAIO, Americo Falcone', 'SAMPAIO, Americo Tadeu Falcone'),
                # ('SUCUPIRA, Elizabeth Furtado','FURTADO, Maria Elizabeth Sucupira'),
                ]


def converter_lista_set(lista):
    set1 = set(lista)
    return set1


def jaccard_similarity(set1, set2):
    '''
    Recebe dois conjuntos como entradas e retorna a similaridade Jaccard entre eles. 
    1. calcula a interseção dos dois conjuntos usando a função de interseção e, 
    2. calcula a união dos dois conjuntos usando a função de união. 
    3. retorna a razão entre o comprimento da interseção e o comprimento da união, que é a similaridade de Jaccard.
    '''
    intersection = set1.intersection(set2)
    union        = set1.union(set2)
    return len(intersection) / len(union)


def similares(lista_autores, lista_grupo, limite_jarowinkler, distancia_levenshtein, lista_extra):
    """Função para aplicar padronização no nome de autor da lista de pesquisadores e buscar similaridade na lista de coautores
     Recebe: Lista de pesquisadores do grupo em análise gerada pela lista de nomes dos coautores das publicações em análise
    Utiliza: get_jaro_distance(), editdistance()
    Retorna: Lista de autores com fusão de nomes cuja similaridade esteja dentro dos limites definidos nesta função
      Autor: Marcos Aires (Fev.2022)
      
    Refazer: Inserir crítica de, mantendo sequência ordem alfabética, retornar no final nome mais extenso em caso de similaridade;
    """
    from pyjarowinkler.distance import get_jaro_distance
    from IPython.display import clear_output
    import editdistance
    import numpy as np
    import time
    
    t0=time.time()
    
    # limite_jarowinkler=0.85
    # distancia_levenshtein=6
    similares_jwl=[]
    similares_regras=[]
    similares=[]
    tempos=[]
    
    count=0
    t1=time.time()
    for i in lista_autores:
        count+=1
        if count > 0:
            tp=time.time()-t1
            tmed=tp/count*2
            tempos.append(tp)
        # print("Analisar similaridades com: ", nome_padronizado)
        
        count1=0
        for nome in lista_autores:
            if count1 > 0:
                resta=len(lista_autores)-count
                print(f'Analisando {count1:3}/{len(lista_autores)} resta analisar {resta:3} nomes. Previsão de término em {np.round(tmed*resta/60,1)} minutos')
            else:
                print(f'Analisando {count1:3}/{len(lista_autores)} resta analisar {len(lista_autores)-count1} nomes.')
            
            t2=time.time()
            count1+=1            

            try:
                similaridade_jarowinkler = get_jaro_distance(i, nome)
                print(f'{i:40} | {nome:40} | Jaro-Winkler: {np.round(similaridade_jarowinkler,2):4} Levenshtein: {editdistance.eval(i, nome)}')
                similaridade_levenshtein = editdistance.eval(i, nome)

                # inferir similaridade para nomes que estejam acima do limite ponderado definido, mas não idênticos e não muito distantes em edição
                if  similaridade_jarowinkler > limite_jarowinkler and similaridade_jarowinkler!=1 and similaridade_levenshtein < distancia_levenshtein:
                    # Crítica no nome mais extenso como destino no par (origem, destino)
                    
                    similares_jwl.append((i,nome))

            except:
                pass

            clear_output(wait=True)
    
    # Conjunto de regras de validação de similaridade
    # Monta uma lista de nomes a serem retirados antes de montar a lista de troca
    trocar=[]
    retirar=[]
    for i in similares_jwl:
        sobrenome_i = i[0].split(',')[0]
        sobrenome_j = i[1].split(',')[0]

        try:
            iniciais_i  = iniciais_nome(i[0]).split(',')[1].strip()
        except:
            iniciais_i  = ''

        try:
            iniciais_j  = iniciais_nome(i[1]).split(',')[1].strip()
        except:
            iniciais_j  = ''

        try:
            primnome_i = i[0].split(',')[1].strip().split(' ')[0].strip()
        except:
            primnome_i = ''

        try:
            primnome_j = i[1].split(',')[1].strip().split(' ')[0].strip()
        except:
            primnome_j = ''    

        try:
            inicial_i = i[0].split(',')[1].strip()[0]
        except:
            inicial_i = ''

        try:
            resto_i   = i[0].split(',')[1].strip().split(' ')[0][1:]
        except:
            resto_i   = ''

        try:
            inicial_j = i[1].split(',')[1].strip()[0]
        except:
            inicial_j = ''

        try:
            resto_j   = i[1].split(',')[1].strip().split(' ')[0][1:]
        except:
            resto_j = ''

        # Se a distância de edição entre os sobrenomes
        if editdistance.eval(sobrenome_i, sobrenome_j) > 2 or inicial_i!=inicial_j:
            retirar.append(i)
        else:
            if primnome_i!=primnome_j and len(primnome_i)>1:
                retirar.append(i)
            if primnome_i!=primnome_j and len(primnome_i)>1 and len(primnome_j)>1:
                retirar.append(i)
            if resto_i!=resto_j and resto_i!='':
                retirar.append(i)
            if len(i[1]) < len(i[0]):
                retirar.append(i)
            if len(iniciais_i) != len(iniciais_j):
                retirar.append(i)

    for i in similares_jwl:
        if i not in retirar:
            trocar.append(i)

        if iniciais_nome(i[0]) in iniciais_nome(i[1]) and len(i[0]) < len(i[1]):
            trocar.append(i)

        if iniciais_nome(i[0]) == iniciais_nome(i[1]) and len(i[0]) < len(i[1]):
             trocar.append(i)
    
    trocar=trocar+lista_extra
    trocar.sort()
    
    return trocar



def extrair_variantes(df_dadosgrupo):
    ''' Utiliza campo de Nome em Citações do currículo como filtro para obter variantes do nome de cada membro
     Recebe: Dataframe com os dados brutos do grupo de pesquisa agrupados; lista de nomes de pesquisadores de interesse
    Retorna: Lista de tuplas com pares a serem trocados da variante pelo nome padronizado na forma (origem, destino)
    '''
    filtro1   = 'Nome'
    lista_nomes = df_dadosgrupo[(df_dadosgrupo.ROTULOS == filtro1)]['CONTEUDOS'].values

    variantes=[]
    filtro='Nome em citações bibliográficas'
    variantes=df_dadosgrupo[(df_dadosgrupo.ROTULOS == filtro)]['CONTEUDOS'].to_list()

    trocar=[]
    for j in range(len(variantes)):
        padrao_destino = padronizar_nome(lista_nomes[j])
        trocar.append((lista_nomes[j], padrao_destino))
        for k in variantes[j]:
            padrao_origem = padronizar_nome(k)
            trocar.append((k, padrao_destino))
            trocar.append((padrao_origem, padrao_destino))
    
    return trocar

    Organizar lista de autores e iniciais de nomes
    PROBLEMA: não está quebrando no número de artigos 1115, mas sim em apenas 2 e última vazia

In [ ]:
## Montar a lista de autores a partir da lista de strings limpa com nomes de autores
def organizar_nomes(str_autores_artigo):
    erros_organizar  = []
    lista_organizada = []
    partes_nomes = str_autores_artigo.split(', ')
    n       = len(partes_nomes)
    pares   = range(0,n+1,2)
    impares = range(1,n+1,2)
    try:
        for i,j in zip(pares,impares):
            nome=[]
            try:
                nomes_ordenado = str(partes_nomes[j].lower()+' '+partes_nomes[i].lower()).replace('  ',' ').strip()
            except:
                if len(pares) > len(impares):
                    nomes_ordenado = str(partes_nomes[j].lower()).replace('  ',' ').strip()
                else:
                    nomes_ordenado = str(partes_nomes[i].lower()).replace('  ',' ').strip()
            lista_organizada.append(nomes_ordenado)
    
    except Exception as e1:
        print('Erro ao montar listas de nomes de autor:',e1)
        erros_organizar.append((m,str_autores_artigo))

    return lista_organizada, erros_organizar

## Quebrar um nome em seu sobrenome seguido das partes de nome
def quebrar_partesnomes(nome):
    padrao     = padronizar_nome(nome).lower()
    sobrenome  = padrao.split(',')[0].strip()
    restonomes = padrao.split(',')[1].strip().split(' ')
    try:
        partenome1 = restonomes[0].strip()
    except:
        partenome1 = np.NaN
    try:
        partenome2 = restonomes[1].strip()
    except:
        partenome2 = np.NaN
    try:
        partenome3 = restonomes[2].strip()
    except:
        partenome3 = np.NaN        
    # print(f'{sobrenome:15} | {partenome1:1} | {partenome2:1} | {partenome3}')
    
    return sobrenome, partenome1, partenome2, partenome3


## Quebrar um nome em seu sobrenome seguido as iniciais das partes de nome
def quebrar_iniciais(nome):
    padrao = iniciais_nome(nome).lower()
    sobrenome  = padrao.split(',')[0].strip()
    restonomes = padrao.split(',')[1].strip().split(' ')

    try:
        partenome1 = restonomes[0].strip()
    except:
        partenome1 = np.NaN
    try:
        partenome2 = restonomes[1].strip()
    except:
        partenome2 = np.NaN
    try:
        partenome3 = restonomes[2].strip()
    except:
        partenome3 = np.NaN        
    # print(f'{sobrenome:15} | {partenome1:1} | {partenome2:1} | {partenome3}')
    
    return sobrenome, partenome1, partenome2, partenome3


## compilar padrão regular expression para buscar ocorência de duas das quatro partes de nome dentro de janela de no máximo 2 palavras de distância
def compilar_partes(sobrenome, partenome1, partenome2, partenome3):
    return re.compile(r'\b({0}|{1}|{2}|{3})(?:\W+\w+){{0,2}}?\W+({1}&{2}|{2}&{3}|{0}&{1}|{0}&{3})\b'.format(sobrenome, partenome1, partenome2, partenome3), flags=re.IGNORECASE)

def compilar_iniciais(sobrenome, inicial1, inicial2, inicial3):
    return re.compile(r'\b({0})(?:\W+\w+){{0,1}}?\W+({1}|{2}|{3})\b'.format(sobrenome, inicial1, inicial2, inicial3), flags=re.IGNORECASE)